![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.pretrained import  PretrainedPipeline

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 3.4.0
sparknlp_jsl.version : 3.4.0
Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.0 
 Master 
 spark://10.139.64.6:7077 
 AppName 
 Databricks Shell

## Posology Releation Extraction

This is a demonstration of using SparkNLP for extracting posology relations. The following relatios are supported:

DRUG-DOSAGE
DRUG-FREQUENCY
DRUG-ADE (Adversed Drug Events)
DRUG-FORM
DRUG-ROUTE
DRUG-DURATION
DRUG-REASON
DRUG=STRENGTH

The model has been validated against the posology dataset described in (Magge, Scotch, & Gonzalez-Hernandez, 2018).

| Relation | Recall | Precision | F1 | F1 (Magge, Scotch, & Gonzalez-Hernandez, 2018) |
| --- | --- | --- | --- | --- |
| DRUG-ADE | 0.66 | 1.00 | **0.80** | 0.76 |
| DRUG-DOSAGE | 0.89 | 1.00 | **0.94** | 0.91 |
| DRUG-DURATION | 0.75 | 1.00 | **0.85** | 0.92 |
| DRUG-FORM | 0.88 | 1.00 | **0.94** | 0.95* |
| DRUG-FREQUENCY | 0.79 | 1.00 | **0.88** | 0.90 |
| DRUG-REASON | 0.60 | 1.00 | **0.75** | 0.70 |
| DRUG-ROUTE | 0.79 | 1.00 | **0.88** | 0.95* |
| DRUG-STRENGTH | 0.95 | 1.00 | **0.98** | 0.97 |


*Magge, Scotch, Gonzalez-Hernandez (2018) collapsed DRUG-FORM and DRUG-ROUTE into a single relation.

In [0]:
import functools 
import numpy as np
from scipy import spatial
import pyspark.sql.functions as F
import pyspark.sql.types as T
from sparknlp.base import *

**Build pipeline using SparNLP pretrained models and the relation extration model optimized for posology**.
 
 The precision of the RE model is controlled by "setMaxSyntacticDistance(4)", which sets the maximum syntactic distance between named entities to 4. A larger value will improve recall at the expense at lower precision. A value of 4 leads to literally perfect precision (i.e. the model doesn't produce any false positives) and reasonably good recall.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

**Create a light pipeline for annotating free text**

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 
12 units of insulin lispro with meals , and metformin 1000 mg two times a day.
"""

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
results[0].keys()

Out[6]: dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
results[0]['ner_chunks']

Out[7]: [Annotation(chunk, 28, 33, 1 unit, {'chunk': '0', 'confidence': '0.8881', 'ner_source': 'ner_chunks', 'entity': 'DOSAGE', 'sentence': '0'}),
 Annotation(chunk, 38, 42, Advil, {'chunk': '1', 'confidence': '0.9851', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '0'}),
 Annotation(chunk, 44, 53, for 5 days, {'chunk': '2', 'confidence': '0.5786', 'ner_source': 'ner_chunks', 'entity': 'DURATION', 'sentence': '0'}),
 Annotation(chunk, 96, 101, 1 unit, {'chunk': '3', 'confidence': '0.8848', 'ner_source': 'ner_chunks', 'entity': 'DOSAGE', 'sentence': '1'}),
 Annotation(chunk, 106, 114, Metformin, {'chunk': '4', 'confidence': '0.9997', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '1'}),
 Annotation(chunk, 116, 120, daily, {'chunk': '5', 'confidence': '0.9998', 'ner_source': 'ner_chunks', 'entity': 'FREQUENCY', 'sentence': '1'}),
 Annotation(chunk, 190, 197, 40 units, {'chunk': '6', 'confidence': '0.92544997', 'ner_source': 'ner_chunks', 'entity': 'DOSAGE', 'sentence': '2'}),
 Annotation(chunk, 202, 217, insulin glargine, {'chunk': '7', 'confidence': '0.7019', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '2'}),
 Annotation(chunk, 219, 226, at night, {'chunk': '8', 'confidence': '0.996', 'ner_source': 'ner_chunks', 'entity': 'FREQUENCY', 'sentence': '2'}),
 Annotation(chunk, 231, 238, 12 units, {'chunk': '9', 'confidence': '0.87450004', 'ner_source': 'ner_chunks', 'entity': 'DOSAGE', 'sentence': '2'}),
 Annotation(chunk, 243, 256, insulin lispro, {'chunk': '10', 'confidence': '0.71625', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '2'}),
 Annotation(chunk, 258, 267, with meals, {'chunk': '11', 'confidence': '0.9977', 'ner_source': 'ner_chunks', 'entity': 'FREQUENCY', 'sentence': '2'}),
 Annotation(chunk, 275, 283, metformin, {'chunk': '12', 'confidence': '0.9655', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '2'}),
 Annotation(chunk, 285, 291, 1000 mg, {'chunk': '13', 'confidence': '0.85745', 'ner_source': 'ner_chunks', 'entity': 'STRENGTH', 'sentence': '2'}),
 Annotation(chunk, 293, 307, two times a day, {'chunk': '14', 'confidence': '0.7618', 'ner_source': 'ner_chunks', 'entity': 'FREQUENCY', 'sentence': '2'})]

In [0]:
results[0]['relations']

Out[8]: [Annotation(category, 28, 42, DOSAGE-DRUG, {'chunk2': 'Advil', 'confidence': '1.0', 'entity2_end': '42', 'chunk1': '1 unit', 'entity2_begin': '38', 'entity1': 'DOSAGE', 'entity1_begin': '28', 'entity1_end': '33', 'entity2': 'DRUG'}),
 Annotation(category, 38, 53, DRUG-DURATION, {'chunk2': 'for 5 days', 'confidence': '1.0', 'entity2_end': '53', 'chunk1': 'Advil', 'entity2_begin': '44', 'entity1': 'DRUG', 'entity1_begin': '38', 'entity1_end': '42', 'entity2': 'DURATION'}),
 Annotation(category, 96, 114, DOSAGE-DRUG, {'chunk2': 'Metformin', 'confidence': '1.0', 'entity2_end': '114', 'chunk1': '1 unit', 'entity2_begin': '106', 'entity1': 'DOSAGE', 'entity1_begin': '96', 'entity1_end': '101', 'entity2': 'DRUG'}),
 Annotation(category, 106, 120, DRUG-FREQUENCY, {'chunk2': 'daily', 'confidence': '1.0', 'entity2_end': '120', 'chunk1': 'Metformin', 'entity2_begin': '116', 'entity1': 'DRUG', 'entity1_begin': '106', 'entity1_end': '114', 'entity2': 'FREQUENCY'}),
 Annotation(category, 190, 217, DOSAGE-DRUG, {'chunk2': 'insulin glargine', 'confidence': '1.0', 'entity2_end': '217', 'chunk1': '40 units', 'entity2_begin': '202', 'entity1': 'DOSAGE', 'entity1_begin': '190', 'entity1_end': '197', 'entity2': 'DRUG'}),
 Annotation(category, 202, 226, DRUG-FREQUENCY, {'chunk2': 'at night', 'confidence': '1.0', 'entity2_end': '226', 'chunk1': 'insulin glargine', 'entity2_begin': '219', 'entity1': 'DRUG', 'entity1_begin': '202', 'entity1_end': '217', 'entity2': 'FREQUENCY'}),
 Annotation(category, 231, 256, DOSAGE-DRUG, {'chunk2': 'insulin lispro', 'confidence': '1.0', 'entity2_end': '256', 'chunk1': '12 units', 'entity2_begin': '243', 'entity1': 'DOSAGE', 'entity1_begin': '231', 'entity1_end': '238', 'entity2': 'DRUG'}),
 Annotation(category, 243, 267, DRUG-FREQUENCY, {'chunk2': 'with meals', 'confidence': '1.0', 'entity2_end': '267', 'chunk1': 'insulin lispro', 'entity2_begin': '258', 'entity1': 'DRUG', 'entity1_begin': '243', 'entity1_end': '256', 'entity2': 'FREQUENCY'}),
 Annotation(category, 275, 291, DRUG-STRENGTH, {'chunk2': '1000 mg', 'confidence': '1.0', 'entity2_end': '291', 'chunk1': 'metformin', 'entity2_begin': '285', 'entity1': 'DRUG', 'entity1_begin': '275', 'entity1_end': '283', 'entity2': 'STRENGTH'}),
 Annotation(category, 275, 307, DRUG-FREQUENCY, {'chunk2': 'two times a day', 'confidence': '1.0', 'entity2_end': '307', 'chunk1': 'metformin', 'entity2_begin': '293', 'entity1': 'DRUG', 'entity1_begin': '275', 'entity1_end': '283', 'entity2': 'FREQUENCY'})]

**Show extracted relations**

In [0]:
for rel in results[0]["relations"]:
    print("{}({}={} - {}={})".format(
        rel.result,
        rel.metadata['entity1'], 
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['chunk2']
    ))

DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Advil)
DRUG-DURATION(DRUG=Advil - DURATION=for 5 days)
DOSAGE-DRUG(DOSAGE=1 unit - DRUG=Metformin)
DRUG-FREQUENCY(DRUG=Metformin - FREQUENCY=daily)
DOSAGE-DRUG(DOSAGE=40 units - DRUG=insulin glargine)
DRUG-FREQUENCY(DRUG=insulin glargine - FREQUENCY=at night)
DOSAGE-DRUG(DOSAGE=12 units - DRUG=insulin lispro)
DRUG-FREQUENCY(DRUG=insulin lispro - FREQUENCY=with meals)
DRUG-STRENGTH(DRUG=metformin - STRENGTH=1000 mg)
DRUG-FREQUENCY(DRUG=metformin - FREQUENCY=two times a day)

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df


rel_df = get_relations_df (results)

rel_df

Out[10]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 DOSAGE-DRUG 
 DOSAGE 
 28 
 33 
 1 unit 
 DRUG 
 38 
 42 
 Advil 
 1.0 
 
 
 1 
 DRUG-DURATION 
 DRUG 
 38 
 42 
 Advil 
 DURATION 
 44 
 53 
 for 5 days 
 1.0 
 
 
 2 
 DOSAGE-DRUG 
 DOSAGE 
 96 
 101 
 1 unit 
 DRUG 
 106 
 114 
 Metformin 
 1.0 
 
 
 3 
 DRUG-FREQUENCY 
 DRUG 
 106 
 114 
 Metformin 
 FREQUENCY 
 116 
 120 
 daily 
 1.0 
 
 
 4 
 DOSAGE-DRUG 
 DOSAGE 
 190 
 197 
 40 units 
 DRUG 
 202 
 217 
 insulin glargine 
 1.0 
 
 
 5 
 DRUG-FREQUENCY 
 DRUG 
 202 
 217 
 insulin glargine 
 FREQUENCY 
 219 
 226 
 at night 
 1.0 
 
 
 6 
 DOSAGE-DRUG 
 DOSAGE 
 231 
 238 
 12 units 
 DRUG 
 243 
 256 
 insulin lispro 
 1.0 
 
 
 7 
 DRUG-FREQUENCY 
 DRUG 
 243 
 256 
 insulin lispro 
 FREQUENCY 
 258 
 267 
 with meals 
 1.0 
 
 
 8 
 DRUG-STRENGTH 
 DRUG 
 275 
 283 
 metformin 
 STRENGTH 
 285 
 291 
 1000 mg 
 1.0 
 
 
 9 
 DRUG-FREQUENCY 
 DRUG 
 275 
 283 
 metformin 
 FREQUENCY 
 293 
 307 
 two times a day 
 1.0

In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df



Out[11]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 DURATION-DRUG 
 DURATION 
 493 
 500 
 five-day 
 DRUG 
 512 
 522 
 amoxicillin 
 1.0 
 
 
 1 
 DRUG-DURATION 
 DRUG 
 681 
 693 
 dapagliflozin 
 DURATION 
 695 
 708 
 for six months 
 1.0 
 
 
 2 
 DRUG-ROUTE 
 DRUG 
 1940 
 1946 
 insulin 
 ROUTE 
 1948 
 1951 
 drip 
 1.0 
 
 
 3 
 DOSAGE-DRUG 
 DOSAGE 
 2255 
 2262 
 40 units 
 DRUG 
 2267 
 2282 
 insulin glargine 
 1.0 
 
 
 4 
 DRUG-FREQUENCY 
 DRUG 
 2267 
 2282 
 insulin glargine 
 FREQUENCY 
 2284 
 2291 
 at night 
 1.0 
 
 
 5 
 DOSAGE-DRUG 
 DOSAGE 
 2295 
 2302 
 12 units 
 DRUG 
 2307 
 2320 
 insulin lispro 
 1.0 
 
 
 6 
 DRUG-FREQUENCY 
 DRUG 
 2307 
 2320 
 insulin lispro 
 FREQUENCY 
 2322 
 2331 
 with meals 
 1.0 
 
 
 7 
 DRUG-STRENGTH 
 DRUG 
 2339 
 2347 
 metformin 
 STRENGTH 
 2349 
 2355 
 1000 mg 
 1.0 
 
 
 8 
 DRUG-FREQUENCY 
 DRUG 
 2339 
 2347 
 metformin 
 FREQUENCY 
 2357 
 2371 
 two times a day 
 1.0

**Visualization of Extracted Relations**

In [0]:
text = """
The patient was prescribed 1 unit of Advil for 5 days after meals. The patient was also 
given 1 unit of Metformin daily.
He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine , 
12 units of insulin lispro with meals , and metformin two times a day.
"""

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
from sparknlp_display import RelationExtractionVisualizer


visualizer = RelationExtractionVisualizer()
vis = visualizer.display(results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

The patient was prescribed 1 unit DOSAGE of Advil DRUG for 5 days DURATION after meals. The patient was also given 1 unit DOSAGE of Metformin DRUG daily FREQUENCY . He was seen by the endocrinology service and she was discharged on 40 units DOSAGE of insulin glargine DRUG , 12 units DOSAGE of insulin lispro DRUG with meals FREQUENCY , and metformin DRUG two times a day FREQUENCY . <polyline fill="none" points="338.0,155.0 337.01020408163265,153.7960798646822 336.02040816326524,152.59312871337622 335.030612244898,151.39211553009378 334.0408163265306,150.19400929884657 333.05102040816325,148.99977900364644 332.06122448979585,147.81039362850512 331.07142857142867,146.62682215743445 330.08163265306126,145.45003357444605 329.09183673469386,144.28099686355176 328.1020408163265,143.12068100876337 327.11224489795916,141.97005499409258 326.12244897959187,140.83008780355124 325.1326530612245,139.70174842115105 324.14285714285717,138.5860058309038 323.15306122448993,137.48382901682123 322.1632653061225,136.39618696291512 321.1734693877552,135.32404865319725 320.18367346938777,134.2683830716793 319.1938775510205,133.23015920237316 318.204081632653,132.2103460292905 317.21428571428567,131.20991253644314 316.2244897959184,130.22982770784282 315.2346938775511,129.2710605275013 314.2448979591837,128.33457997943034 313.2551020408164,127.42135504764173 312.26530612244903,126.53235471614718 311.2755102040816,125.6685479689585 310.28571428571433,124.83090379008749 309.295918367347,124.02039116354581 308.3061224489796,123.2379790733453 307.3163265306123,122.48463650349771 306.32653061224494,121.76133243801479 305.33673469387753,121.06903586090831 304.3469387755102,120.40871575619002 303.3571428571429,119.78134110787173 302.36734693877554,119.18788089996517 301.3775510204082,118.62930411648207 300.38775510204084,118.10657974143427 299.3979591836735,117.62067675883348 298.40816326530614,117.17256415269148 297.4183673469388,116.76321090702004 296.42857142857144,116.39358600583091 295.4387755102041,116.06465843313586 294.44897959183675,115.77739717294665 293.4591836734694,115.53277120927505 292.46938775510205,115.33174952613282 291.4795918367347,115.17530110753174 290.48979591836735,115.06439493748353 289.5,115.0 289.5,115.0 288.51020408163265,114.9827622844223 287.52040816326524,115.01203580140927 286.530612244898,115.08685156694916 285.54081632653066,115.20624059703015 284.55102040816325,115.36923390764053 283.56122448979585,115.57486251476851 282.5714285714286,115.82215743440234 281.58163265306126,116.11014968253025 280.5918367346939,116.43787027514047 279.60204081632645,116.80435022822121 278.61224489795916,117.20862055776078 277.62244897959175,117.64971227974738 276.6326530612244,118.12665641016923 275.6428571428571,118.63848396501459 274.6530612244898,119.18422596027166 273.66326530612247,119.7629134119287 272.6734693877551,120.37357733597398 271.6836734693877,121.01524874839565 270.69387755102036,121.68695866518203 269.704081632653,122.3877381023213 268.7142857142856,123.11661807580174 267.7244897959184,123.87262960161156 266.73469387755097,124.654803695739 265.7448979591837,125.46217137417233 264.75510204081627,126.29376365289971 263.7653061224489,127.14861154790944 262.7755102040816,128.02574607518974 261.7857142857143,128.92419825072886 260.7959183673469,129.842999090515 259.8061224489796,130.7811796105364 258.8163265306123,131.73777082678137 257.8265306122449,132.71180375523804 256.83673469387753,133.7023094118947 255.84693877551018,134.7083188127396 254.85714285714283,135.72886297376095 253.86734693877548,136.76297291094698 252.87755102040813,137.80967964028594 251.88775510204079,138.86801417776604 250.89795918367344,139.9370075393756 249.90816326530611,141.01569074110276 248.9183673469388,142.1030947989358 247.92857142857144,143.19825072886297 246.93877551020407,144.30018954687245 245.94897959183675,145.40794226895255 244.9591836734694,146.52053991109145 243.96938775510205,147.63701348927742 242.97959183673467,148.75639401949866 241.98979

## ADE Relation Extraction

We can extract relations between `DRUG` and adverse reactions `ADE` caused by them. Also we can find if an adverse event is caused by a drug or not.

`1`: Shows the adverse event and drug entities are related.

`0`: Shows the adverse event and drug entities are not related.

Now we will first use `ner_ade_clinical` NER model and detect `DRUG` and `ADE` entities. Then we ca find the relations between them by using `re_ade_clinical` model. We can also use the same pipeline elemnets rest of them we created above.

In [0]:
ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")  

reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_model = ade_pipeline.fit(empty_data)

ner_ade_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
re_ade_clinical download started this may take some time.
Approximate size to download 10.9 MB
[ | ][ / ][OK!]

We will create a LightPipeline for annotation.

In [0]:
ade_lmodel = LightPipeline(ade_model)

In [0]:
text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

ade_results = ade_lmodel.fullAnnotate(text)

In [0]:
ade_results[0].keys()

Out[21]: dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
ade_results[0]["ner_chunks"]

Out[22]: [Annotation(chunk, 25, 32, naproxen, {'chunk': '0', 'confidence': '0.9954', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '0'}),
 Annotation(chunk, 87, 95, oxaprozin, {'chunk': '1', 'confidence': '0.5896', 'ner_source': 'ner_chunks', 'entity': 'DRUG', 'sentence': '0'}),
 Annotation(chunk, 137, 148, tense bullae, {'chunk': '2', 'confidence': '0.7753', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'}),
 Annotation(chunk, 154, 210, cutaneous fragility on the face and the back of the hands, {'chunk': '3', 'confidence': '0.7352181', 'ner_source': 'ner_chunks', 'entity': 'ADE', 'sentence': '0'})]

**Lets show the `ADE`-`DRUG` relations by using pandas dataframe.**

In [0]:
print(text)

rel_df = get_relations_df (ade_results)

rel_df


A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.
Out[23]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 1 
 DRUG 
 25 
 32 
 naproxen 
 ADE 
 137 
 148 
 tense bullae 
 1.0 
 
 
 1 
 1 
 DRUG 
 25 
 32 
 naproxen 
 ADE 
 154 
 210 
 cutaneous fragility on the face and the back of the hands 
 0.99999964 
 
 
 2 
 1 
 DRUG 
 87 
 95 
 oxaprozin 
 ADE 
 137 
 148 
 tense bullae 
 1.0 
 
 
 3 
 1 
 DRUG 
 87 
 95 
 oxaprozin 
 ADE 
 154 
 210 
 cutaneous fragility on the face and the back of the hands 
 1.0

In [0]:
text = "I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor."

ade_results = ade_lmodel.fullAnnotate(text)

rel_df = get_relations_df (ade_results)

rel_df

Out[24]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 1 
 ADE 
 14 
 20 
 fatigue 
 DRUG 
 82 
 88 
 Lipitor 
 0.9996617 
 
 
 1 
 0 
 ADE 
 14 
 20 
 fatigue 
 DRUG 
 124 
 128 
 Zocor 
 0.9952187 
 
 
 2 
 1 
 ADE 
 23 
 35 
 muscle cramps 
 DRUG 
 82 
 88 
 Lipitor 
 0.9999827 
 
 
 3 
 0 
 ADE 
 23 
 35 
 muscle cramps 
 DRUG 
 124 
 128 
 Zocor 
 0.91462934 
 
 
 4 
 1 
 ADE 
 38 
 44 
 anxiety 
 DRUG 
 82 
 88 
 Lipitor 
 0.7636133 
 
 
 5 
 0 
 ADE 
 38 
 44 
 anxiety 
 DRUG 
 124 
 128 
 Zocor 
 0.9999691 
 
 
 6 
 1 
 ADE 
 47 
 55 
 agression 
 DRUG 
 82 
 88 
 Lipitor 
 0.99999833 
 
 
 7 
 0 
 ADE 
 47 
 55 
 agression 
 DRUG 
 124 
 128 
 Zocor 
 0.99781835 
 
 
 8 
 1 
 ADE 
 61 
 67 
 sadness 
 DRUG 
 82 
 88 
 Lipitor 
 1.0 
 
 
 9 
 0 
 ADE 
 61 
 67 
 sadness 
 DRUG 
 124 
 128 
 Zocor 
 0.9999572

**Visualization of relations**

In [0]:
visualizer = RelationExtractionVisualizer()
vis = visualizer.display(results[0], 'relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

I experienced fatigue ADE , muscle cramps ADE , anxiety ADE , agression ADE and sadness ADE after taking Lipitor DRUG but no more adverse after passing Zocor DRUG . <polyline fill="none" points="479.0,155.0 479.27551020408157,154.33673469387753 479.5510204081632,153.6734693877551 479.82653061224494,153.01020408163268 480.1020408163265,152.3469387755102 480.37755102040813,151.68367346938777 480.65306122448976,151.0204081632653 480.92857142857156,150.35714285714292 481.20408163265313,149.69387755102042 481.47959183673476,149.03061224489798 481.7551020408162,148.36734693877554 482.03061224489784,147.70408163265307 482.3061224489795,147.0408163265306 482.5816326530612,146.3775510204082 482.85714285714283,145.71428571428572 483.1326530612246,145.05102040816328 483.40816326530614,144.38775510204084 483.6836734693878,143.7244897959184 483.9591836734694,143.0612244897959 484.234693877551,142.3979591836735 484.5102040816326,141.734693877551 484.7857142857142,141.07142857142856 485.06122448979596,140.40816326530614 485.33673469387753,139.74489795918367 485.6122448979592,139.08163265306123 485.8877551020408,138.41836734693877 486.16326530612236,137.75510204081633 486.4387755102041,137.09183673469389 486.7142857142858,136.42857142857144 486.98979591836735,135.765306122449 487.2653061224489,135.10204081632654 487.54081632653066,134.4387755102041 487.81632653061223,133.77551020408163 488.09183673469386,133.11224489795921 488.3673469387755,132.44897959183675 488.64285714285717,131.7857142857143 488.9183673469388,131.12244897959184 489.19387755102036,130.4591836734694 489.46938775510205,129.79591836734693 489.7448979591837,129.13265306122452 490.0204081632653,128.46938775510205 490.295918367347,127.80612244897961 490.57142857142856,127.14285714285717 490.84693877551024,126.4795918367347 491.1224489795918,125.81632653061226 491.3979591836735,125.15306122448982 491.6734693877551,124.48979591836735 491.94897959183675,123.8265306122449 492.2244897959184,123.16326530612245 492.5,122.5 492.5,122.5 492.7755102040816,121.83673469387755 493.05102040816325,121.17346938775509 493.32653061224494,120.51020408163266 493.6020408163265,119.84693877551021 493.8775510204082,119.18367346938776 494.15306122448976,118.5204081632653 494.42857142857156,117.85714285714289 494.70408163265313,117.19387755102042 494.9795918367347,116.53061224489795 495.25510204081627,115.86734693877548 495.53061224489784,115.20408163265304 495.8061224489795,114.54081632653057 496.0816326530612,113.87755102040816 496.35714285714283,113.21428571428571 496.6326530612246,112.55102040816328 496.9081632653062,111.88775510204083 497.1836734693878,111.22448979591837 497.4591836734694,110.56122448979592 497.73469387755097,109.89795918367346 498.01020408163254,109.23469387755101 498.2857142857142,108.57142857142856 498.56122448979596,107.90816326530613 498.8367346938776,107.24489795918367 499.11224489795916,106.58163265306122 499.3877551020408,105.91836734693875 499.66326530612236,105.25510204081633 499.93877551020404,104.59183673469387 500.2142857142858,103.92857142857144 500.48979591836735,103.26530612244898 500.765306122449,102.60204081632654 501.0408163265306,101.9387755102041 501.3163265306123,101.27551020408163 501.59183673469386,100.61224489795917 501.8673469387755,99.94897959183672 502.14285714285717,99.28571428571428 502.4183673469388,98.62244897959184 502.6938775510205,97.9591836734694 502.96938775510205,97.29591836734693 503.24489795918373,96.63265306122449 503.5204081632653,95.96938775510205 503.795918367347,95.3061224489796 504.0714285714286,94.64285714285714 504.3469387755102,93.9795918367347 504.6224489795918,93.31632653061224 504.8979591836735,92.6530612244898 505.1734693877551,91.98979591836735 505.44897959183675,91.3265306122449 505.7244897959183,90.66326530612244 506.0,90.0" stroke="#e4815e" stroke-width="1" /> 0 <polyline fill="none" points="494.0,155.0 494.9591836734693,154.33673469387753 495.9183673469387,153.6734693877551 496.8775510204082,153.01020408163268 497.8367346938

## Clinical RE

**The set of relations defined in the 2010 i2b2 relation challenge**

TrIP: A certain treatment has improved or cured a medical problem (eg, ‘infection resolved with antibiotic course’)

TrWP: A patient's medical problem has deteriorated or worsened because of or in spite of a treatment being administered (eg, ‘the tumor was growing despite the drain’)

TrCP: A treatment caused a medical problem (eg, ‘penicillin causes a rash’)

TrAP: A treatment administered for a medical problem (eg, ‘Dexamphetamine for narcolepsy’)

TrNAP: The administration of a treatment was avoided because of a medical problem (eg, ‘Ralafen which is contra-indicated because of ulcers’)

TeRP: A test has revealed some medical problem (eg, ‘an echocardiogram revealed a pericardial effusion’)

TeCP: A test was performed to investigate a medical problem (eg, ‘chest x-ray done to rule out pneumonia’)

PIP: Two problems are related to each other (eg, ‘Azotemia presumed secondary to sepsis’)

In [0]:
clinical_ner_tagger = MedicalNerModel()\
    .pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setRelationPairs(["problem-test", "problem-treatment"]) # we can set the possible relation pairs (if not set, all the relations will be calculated)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
re_clinical download started this may take some time.
Approximate size to download 6 MB
[ | ][OK!]

In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.relation!="O"]


Out[26]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 11 
 TrAP 
 PROBLEM 
 617 
 620 
 T2DM 
 TREATMENT 
 626 
 637 
 atorvastatin 
 0.99955326 
 
 
 15 
 TeRP 
 TEST 
 739 
 758 
 Physical examination 
 PROBLEM 
 796 
 810 
 dry oral mucosa 
 0.9994142 
 
 
 19 
 TrWP 
 TEST 
 1246 
 1258 
 blood samples 
 PROBLEM 
 1265 
 1274 
 hemolyzing 
 0.9854173 
 
 
 21 
 TeRP 
 TEST 
 1535 
 1547 
 the anion gap 
 PROBLEM 
 1553 
 1566 
 still elevated 
 0.9965193 
 
 
 26 
 TrAP 
 TEST 
 1838 
 1845 
 analysis 
 PROBLEM 
 1854 
 1880 
 interference from turbidity 
 0.9676019 
 
 
 33 
 TrAP 
 PROBLEM 
 1967 
 1969 
 HTG 
 TREATMENT 
 1976 
 1986 
 a reduction 
 0.9875973 
 
 
 34 
 TrAP 
 PROBLEM 
 1967 
 1969 
 HTG 
 TEST 
 1991 
 2003 
 the anion gap 
 0.9993911

## Clinical Temporal Events RE

Temporal relations, or temporal links (denoted by the TLINK tag), indicate whether and how two EVENTs, two TIME, or an EVENT and a TIME related to each other in the clinical timeline. There are 3 type of relations here and below are some examples of Relations, with square brackets indicating EVENT and TIME connected by a temporal link:

**`BEFORE`**

The patient was given stress dose steroids prior to his surgery. ([stress dose steroids] `BEFORE` [his surgery])

The patient had an undocumented history of possible atrial fibrillation prior to admission. ([possible atrial fibrillation] `BEFORE` [admission])

His nasogastric tube was discontinued on 05-26-98. ([His nasogastric] `BEFORE` [05-26-98])

**`AFTER`**

Before admission, he had another serious concussion. ([admission] `AFTER` [another serious concussion])

On postoperative day No 1, he was started on Percocet. ([Percocet] `AFTER` [postoperative day No 1])


**`OVERLAP`**

She denies any fevers or chills. ([fevers] `OVERLAP` [chills])

The patient's serum creatinine on discharge date, 2012-05-06, was 1.9. ([discharge date] `OVERLAP` [2012-05-06])

His preoperative workup was completed and included a normal white count ([a normal white count] `OVERLAP` [His preoperative workup])

The patient had an undocumented history of possible atrial fibrillation prior to admission. ([possible atrial fibrillation] `OVERLAP` [admission])


| Relation | Recall | Precision | F1 |
| --- | --- | --- | --- |
| OVERLAP | 0.81 | 0.73 | **0.77** |
| BEFORE | 0.85 | 0.88 | **0.86** |
| AFTER | 0.38 | 0.46 | **0.43** |

This RE model works with `ner_events_clinical` NER model and expect the following entities as inputs:

[`OCCURRENCE`,
 `DATE`,
 `DURATION`,
 `EVIDENTIAL`,
 `TEST`,
 `PROBLEM`,
 `TREATMENT`,
 `CLINICAL_DEPT`,
 `FREQUENCY`,
 `TIME`]

In [0]:
events_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setPredictionThreshold(0.9)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][OK!]
re_temporal_events_clinical download started this may take some time.
Approximate size to download 5.9 MB
[ | ][OK!]

In [0]:
events_ner_tagger.getClasses()

Out[28]: ['O',
 'B-OCCURRENCE',
 'B-DATE',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-DURATION',
 'I-DURATION',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'B-TREATMENT',
 'I-TREATMENT',
 'I-DATE',
 'B-TEST',
 'I-TEST',
 'B-CLINICAL_DEPT',
 'I-CLINICAL_DEPT',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'I-EVIDENTIAL',
 'B-TIME',
 'I-TIME']

In [0]:
text ="She is diagnosed as cancer in 1991. Then she was admitted to Mayo Clinic in May 2000 and discharged in October 2001"

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df

Out[29]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 BEFORE 
 PROBLEM 
 20 
 25 
 cancer 
 DATE 
 30 
 33 
 1991 
 0.9989215 
 
 
 1 
 AFTER 
 OCCURRENCE 
 49 
 56 
 admitted 
 CLINICAL_DEPT 
 61 
 71 
 Mayo Clinic 
 1.0 
 
 
 2 
 OVERLAP 
 OCCURRENCE 
 49 
 56 
 admitted 
 DATE 
 76 
 83 
 May 2000 
 0.99152386 
 
 
 3 
 BEFORE 
 CLINICAL_DEPT 
 61 
 71 
 Mayo Clinic 
 OCCURRENCE 
 89 
 98 
 discharged 
 1.0 
 
 
 4 
 BEFORE 
 OCCURRENCE 
 89 
 98 
 discharged 
 DATE 
 103 
 114 
 October 2001 
 0.95415086

In [0]:
text ="On 9–28-92, the patient will return for chemotherapy and she will follow up with her primary doctor, for PT and Coumadin dosing on Monday."

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df[(rel_df.relation!="O")&(rel_df.entity1!=rel_df.entity2)]


Out[30]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 OVERLAP 
 DATE 
 3 
 9 
 9–28-92 
 TREATMENT 
 40 
 51 
 chemotherapy 
 1.000000 
 
 
 1 
 OVERLAP 
 DATE 
 3 
 9 
 9–28-92 
 TREATMENT 
 105 
 106 
 PT 
 0.989851 
 
 
 3 
 OVERLAP 
 OCCURRENCE 
 66 
 74 
 follow up 
 CLINICAL_DEPT 
 81 
 98 
 her primary doctor 
 0.998086 
 
 
 4 
 OVERLAP 
 OCCURRENCE 
 66 
 74 
 follow up 
 TREATMENT 
 105 
 106 
 PT 
 0.999761 
 
 
 5 
 OVERLAP 
 OCCURRENCE 
 66 
 74 
 follow up 
 TREATMENT 
 112 
 126 
 Coumadin dosing 
 0.958726 
 
 
 6 
 OVERLAP 
 CLINICAL_DEPT 
 81 
 98 
 her primary doctor 
 TREATMENT 
 105 
 106 
 PT 
 1.000000 
 
 
 7 
 OVERLAP 
 CLINICAL_DEPT 
 81 
 98 
 her primary doctor 
 DATE 
 131 
 136 
 Monday 
 0.984193 
 
 
 9 
 AFTER 
 TREATMENT 
 105 
 106 
 PT 
 DATE 
 131 
 136 
 Monday 
 0.926052 
 
 
 10 
 OVERLAP 
 TREATMENT 
 112 
 126 
 Coumadin dosing 
 DATE 
 131 
 136 
 Monday 
 0.999999

**Admission-Discharge Events Exraction**

We can also extract `ADMISSION` and `DISCHARGE` entities in addition to `ner_events_clinical` model entities using `ner_events_admission_clinical` NER model and expect the following entities as inputs:
   
[`EVIDENTIAL`,
 `OCCURRENCE`,
 `TREATMENT`,
 `DATE`,
 `ADMISSION`,
 `TIME`,
 `FREQUENCY`,
 `CLINICAL_DEPT`,
 `DURATION`,
 `PROBLEM`,
 `TEST`,
 `DISCHARGE`]

In [0]:
events_admission_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_admission_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_admission_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_admission_clinical download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][ — ][OK!]

In [0]:
events_admission_ner_tagger.getClasses()

Out[32]: ['O',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'I-TREATMENT',
 'I-DATE',
 'B-ADMISSION',
 'B-TREATMENT',
 'I-TIME',
 'I-FREQUENCY',
 'B-FREQUENCY',
 'B-OCCURRENCE',
 'I-CLINICAL_DEPT',
 'B-DURATION',
 'I-PROBLEM',
 'I-TEST',
 'B-DATE',
 'B-TIME',
 'I-EVIDENTIAL',
 'I-DURATION',
 'B-DISCHARGE',
 'B-TEST',
 'B-PROBLEM',
 'B-CLINICAL_DEPT']

In [0]:
text ="""She is admitted to The John Hopkins Hospital on Monday with a history of gestational diabetes mellitus diagnosed.  
She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 
12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. 
"""

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]


Out[33]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 AFTER 
 ADMISSION 
 7 
 14 
 admitted 
 CLINICAL_DEPT 
 19 
 43 
 The John Hopkins Hospital 
 0.993847 
 
 
 1 
 OVERLAP 
 ADMISSION 
 7 
 14 
 admitted 
 DATE 
 48 
 53 
 Monday 
 0.991630 
 
 
 2 
 OVERLAP 
 CLINICAL_DEPT 
 19 
 43 
 The John Hopkins Hospital 
 DATE 
 48 
 53 
 Monday 
 1.000000 
 
 
 3 
 OVERLAP 
 DATE 
 48 
 53 
 Monday 
 PROBLEM 
 73 
 101 
 gestational diabetes mellitus 
 0.929829 
 
 
 4 
 OVERLAP 
 CLINICAL_DEPT 
 132 
 156 
 the endocrinology service 
 DATE 
 184 
 193 
 03/02/2018 
 0.999932 
 
 
 5 
 OVERLAP 
 DISCHARGE 
 170 
 179 
 discharged 
 DATE 
 184 
 193 
 03/02/2018 
 1.000000 
 
 
 6 
 AFTER 
 DISCHARGE 
 170 
 179 
 discharged 
 TREATMENT 
 210 
 225 
 insulin glargine 
 0.999233 
 
 
 7 
 AFTER 
 DISCHARGE 
 170 
 179 
 discharged 
 TREATMENT 
 241 
 254 
 insulin lispro 
 0.997469 
 
 
 8 
 OVERLAP 
 DATE 
 184 
 193 
 03/02/2018 
 TREATMENT 
 210 
 225 
 insulin glargine 
 0.999975 
 
 
 9 
 OVERLAP 
 DATE 
 184 
 193 
 03/02/2018 
 TREATMENT 
 241 
 254 
 insulin lispro 
 0.999682 
 
 
 10 
 OVERLAP 
 TREATMENT 
 261 
 269 
 metformin 
 FREQUENCY 
 279 
 293 
 two times a day 
 1.000000 
 
 
 11 
 OVERLAP 
 OCCURRENCE 
 304 
 318 
 close follow-up 
 CLINICAL_DEPT 
 325 
 337 
 endocrinology 
 1.000000 
 
 
 12 
 BEFORE 
 OCCURRENCE 
 304 
 318 
 close follow-up 
 DISCHARGE 
 344 
 352 
 discharge 
 0.998952 
 
 
 13 
 AFTER 
 CLINICAL_DEPT 
 325 
 337 
 endocrinology 
 DISCHARGE 
 344 
 352 
 discharge 
 0.999982

**Test-Result-Date Extraction**

There is another Relation Extraction model to link clinical tests to test results and dates to clinical entities: `re_test_result_date`

In [0]:
trd_ner_tagger = MedicalNerModel()\
    .pretrained("jsl_ner_wip_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")       

trd_re_Model = RelationExtractionModel()\
    .pretrained("re_test_result_date", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    trd_ner_tagger,
    ner_chunker,
    dependency_parser,
    trd_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

jsl_ner_wip_clinical download started this may take some time.
Approximate size to download 14.5 MB
[ | ][ / ][OK!]
re_test_result_date download started this may take some time.
Approximate size to download 9.3 MB
[ | ][ / ][OK!]

In [0]:
text = 'Hospitalized with pneumonia on 11 June 2019, confirmed by a positive PCR of any specimen, evidenced by SPO2 </= 93%.'

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]

Out[35]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 is_date_of 
 Disease_Syndrome_Disorder 
 18 
 26 
 pneumonia 
 Date 
 31 
 42 
 11 June 2019 
 0.998061 
 
 
 3 
 is_finding_of 
 Disease_Syndrome_Disorder 
 18 
 26 
 pneumonia 
 O2_Saturation 
 103 
 114 
 SPO2 </= 93% 
 0.984967 
 
 
 6 
 is_date_of 
 Date 
 31 
 42 
 11 June 2019 
 O2_Saturation 
 103 
 114 
 SPO2 </= 93% 
 0.993791 
 
 
 7 
 is_result_of 
 Test_Result 
 60 
 67 
 positive 
 Test 
 69 
 71 
 PCR 
 1.000000 
 
 
 8 
 is_result_of 
 Test_Result 
 60 
 67 
 positive 
 O2_Saturation 
 103 
 114 
 SPO2 </= 93% 
 1.000000

## Human Phenotype - Gene RE

https://github.com/lasigeBioTM/PGR

Human phenotype-gene relations are fundamental to fully understand the origin of some phenotypic abnormalities and their associated diseases. Biomedical literature is the most comprehensive source of these relations, however, we need Relation Extraction tools to automatically recognize them. We present the Phenotype-Gene Relations (PGR) model, trained on a silver standard corpus of human phenotype and gene annotations and their relations. 

It extracts 2 label: `True` or `False`

In [0]:
pgr_ner_tagger = MedicalNerModel()\
    .pretrained("ner_human_phenotype_gene_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pgr_re_Model = RelationExtractionModel()\
    .pretrained("re_human_phenotype_gene_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["hp-gene",'gene-hp'])\
    .setMaxSyntacticDistance(4)\

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pgr_ner_tagger,
    ner_chunker,
    dependency_parser,
    pgr_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_human_phenotype_gene_clinical download started this may take some time.
Approximate size to download 14 MB
[ | ][ / ][OK!]
re_human_phenotype_gene_clinical download started this may take some time.
Approximate size to download 5.9 MB
[ | ][OK!]

In [0]:
text = "She has a retinal degeneration, hearing loss and renal failure, short stature, \
Mutations in the SH3PXD2B gene coding for the Tks4 protein are responsible for the autosomal recessive."

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!=0)]

rel_df

Out[38]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 1 
 HP 
 32 
 43 
 hearing loss 
 GENE 
 96 
 103 
 SH3PXD2B 
 0.99999976 
 
 
 1 
 1 
 HP 
 49 
 61 
 renal failure 
 GENE 
 96 
 103 
 SH3PXD2B 
 0.9998969 
 
 
 2 
 1 
 HP 
 64 
 76 
 short stature 
 GENE 
 96 
 103 
 SH3PXD2B 
 0.99994695

## Drug-Drug Interaction RE

In clinical application, two or more drugs are often used in combination to achieve conducive results, such as synergistic effect, increasing therapeutic effect and reducing or delaying the occurrence of
drug resistance. However, there is a potential for harmful drug-drug
interactions (DDIs) to occur when two or more drugs are taken at the
same time or at certain interval, which can reduce or invalidate the
efficacy of drugs, and increase toxicity or even cause death. Therefore,
in order to prevent harmful drug-drug interaction (DDI), medical staff
often spend much time in reviewing the relevant drug alert literature
and drug knowledge bases. 

**ref**: *Drug-drug interaction extraction via hybrid neural networks on biomedical literature
https://www-sciencedirect-com.ezproxy.leidenuniv.nl:2443/science/article/pii/S1532046420300605*

In [0]:
ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("ner_chunk")

ddi_re_model = RelationExtractionModel()\
    .pretrained("re_drug_drug_interaction_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setRelationPairs(["drug-drug"])\
    .setMaxSyntacticDistance(4)\

ddi_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_converter,
    dependency_parser,
    ddi_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ddi_model = ddi_pipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
re_drug_drug_interaction_clinical download started this may take some time.
Approximate size to download 4.6 MB
[ | ][OK!]

In [0]:
text='When carbamazepine is withdrawn from the combination therapy, aripiprazole dose should then be reduced. \
If additional adrenergic drugs are to be administered by any route, \
they should be used with caution because the pharmacologically predictable sympathetic effects of Metformin may be potentiated'

lmodel = LightPipeline(ddi_model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)


rel_df

Out[40]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 DDI-advise 
 DRUG 
 5 
 17 
 carbamazepine 
 DRUG 
 62 
 73 
 aripiprazole 
 0.99238014

In [0]:
annotations[0]['ner_chunk']

Out[41]: [Annotation(chunk, 5, 17, carbamazepine, {'chunk': '0', 'confidence': '0.5977', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '0'}),
 Annotation(chunk, 62, 73, aripiprazole, {'chunk': '1', 'confidence': '0.995', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '0'}),
 Annotation(chunk, 118, 133, adrenergic drugs, {'chunk': '2', 'confidence': '0.676', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '1'}),
 Annotation(chunk, 270, 278, Metformin, {'chunk': '3', 'confidence': '0.9998', 'ner_source': 'ner_chunk', 'entity': 'DRUG', 'sentence': '1'})]

## Drug-Protein Interaction RE

Detect interactions between chemical compounds/drugs and genes/proteins using Spark NLP’s `RelationExtractionModel()` by classifying whether a specified semantic relation holds between a chemical and gene entities within a sentence or document. <br/>

The entity labels used during training were derived from the custom NER model created by our team for the **DrugProt** corpus. These include `CHEMICAL` for chemical compounds/drugs, `GENE` for genes/proteins and `GENE_AND_CHEMICAL` for entity mentions of type `GENE` and of type `CHEMICAL` that overlap (such as enzymes and small peptides). <br/>

The relation categories from the **DrugProt** corpus were condensed from 13 categories to 10 categories due to low numbers of examples for certain categories. This merging process involved grouping the `SUBSTRATE_PRODUCT-OF` and `SUBSTRATE` relation categories together and grouping the `AGONIST-ACTIVATOR`, `AGONIST-INHIBITOR` and `AGONIST` relation categories together.

**ref**: *DrugProt corpus: Biocreative VII Track 1 - Text mining drug and chemical-protein interactionshttps://zenodo.org/record/5119892*

In [0]:
drugprot_ner_tagger = MedicalNerModel.pretrained("ner_drugprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

drugprot_re_model = RelationExtractionModel()\
        .pretrained("re_drugprot_clinical", "en", 'clinical/models')\
        .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
        .setOutputCol("relations")\
        .setMaxSyntacticDistance(4)\
        .setPredictionThreshold(0.5)\
        .setRelationPairs(["checmical-gene", "chemical-gene_and_chemical", "gene_and_chemical-gene"]) 

pipeline = Pipeline(stages=[
        documenter, 
        sentencer,
        tokenizer,
        words_embedder, 
        drugprot_ner_tagger,
        ner_converter, 
        pos_tagger, 
        dependency_parser, 
        drugprot_re_model])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_drugprot_clinical download started this may take some time.
Approximate size to download 14 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
re_drugprot_clinical download started this may take some time.
Approximate size to download 9.2 MB
[ | ][ / ][OK!]

In [0]:
text='''Lipid specific activation of the murine P4-ATPase Atp8a1 (ATPase II). The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter. This aminophospholipid "flippase" selectively transports PS to the cytosolic leaflet of the bilayer and is sensitive to vanadate, Ca(2+), and modification by sulfhydryl reagents. Although the flippase has not been positively identified, a subfamily of P-type ATPases has been proposed to function as transporters of amphipaths, including PS and other phospholipids. A candidate PS flippase ATP8A1 (ATPase II), originally isolated from bovine secretory vesicles, is a member of this subfamily based on sequence homology to the founding member of the subfamily, the yeast protein Drs2, which has been linked to ribosomal assembly, the formation of Golgi-coated vesicles, and the maintenance of PS asymmetry. To determine if ATP8A1 has biochemical characteristics consistent with a PS flippase, a murine homologue of this enzyme was expressed in insect cells and purified. The purified Atp8a1 is inactive in detergent micelles or in micelles containing phosphatidylcholine, phosphatidic acid, or phosphatidylinositol, is minimally activated by phosphatidylglycerol or phosphatidylethanolamine (PE), and is maximally activated by PS. The selectivity for PS is dependent upon multiple elements of the lipid structure. Similar to the plasma membrane PS transporter, Atp8a1 is activated only by the naturally occurring sn-1,2-glycerol isomer of PS and not the sn-2,3-glycerol stereoisomer. Both flippase and Atp8a1 activities are insensitive to the stereochemistry of the serine headgroup. Most modifications of the PS headgroup structure decrease recognition by the plasma membrane PS flippase. Activation of Atp8a1 is also reduced by these modifications; phosphatidylserine-O-methyl ester, lysophosphatidylserine, glycerophosphoserine, and phosphoserine, which are not transported by the plasma membrane flippase, do not activate Atp8a1. Weakly translocated lipids (PE, phosphatidylhydroxypropionate, and phosphatidylhomoserine) are also weak Atp8a1 activators. However, N-methyl-phosphatidylserine, which is transported by the plasma membrane flippase at a rate equivalent to PS, is incapable of activating Atp8a1 activity. These results indicate that the ATPase activity of the secretory granule Atp8a1 is activated by phospholipids binding to a specific site whose properties (PS selectivity, dependence upon glycerol but not serine, stereochemistry, and vanadate sensitivity) are similar to, but distinct from, the properties of the substrate binding site of the plasma membrane flippase.'''

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
results[0].keys()

Out[44]: dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [0]:
results[0]["ner_chunks"]

Out[45]: [Annotation(chunk, 33, 48, murine P4-ATPase, {'entity': 'GENE', 'sentence': '0', 'chunk': '0', 'confidence': '0.68700004'}),
 Annotation(chunk, 50, 55, Atp8a1, {'entity': 'GENE', 'sentence': '0', 'chunk': '1', 'confidence': '0.9957'}),
 Annotation(chunk, 58, 66, ATPase II, {'entity': 'GENE', 'sentence': '0', 'chunk': '2', 'confidence': '0.77655'}),
 Annotation(chunk, 114, 131, phosphatidylserine, {'entity': 'CHEMICAL', 'sentence': '1', 'chunk': '3', 'confidence': '0.7679'}),
 Annotation(chunk, 134, 135, PS, {'entity': 'CHEMICAL', 'sentence': '1', 'chunk': '4', 'confidence': '0.9767'}),
 Annotation(chunk, 224, 248, ATP-dependent transporter, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '1', 'chunk': '5', 'confidence': '0.87095'}),
 Annotation(chunk, 256, 282, aminophospholipid "flippase, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '2', 'chunk': '6', 'confidence': '0.5996'}),
 Annotation(chunk, 308, 309, PS, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '7', 'confidence': '0.9969'}),
 Annotation(chunk, 371, 378, vanadate, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '8', 'confidence': '0.9464'}),
 Annotation(chunk, 381, 385, Ca(2+, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '9', 'confidence': '0.5832'}),
 Annotation(chunk, 409, 418, sulfhydryl, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '10', 'confidence': '0.7311'}),
 Annotation(chunk, 443, 450, flippase, {'entity': 'GENE', 'sentence': '3', 'chunk': '11', 'confidence': '0.9773'}),
 Annotation(chunk, 503, 516, P-type ATPases, {'entity': 'GENE', 'sentence': '3', 'chunk': '12', 'confidence': '0.73015'}),
 Annotation(chunk, 589, 590, PS, {'entity': 'CHEMICAL', 'sentence': '3', 'chunk': '13', 'confidence': '0.9953'}),
 Annotation(chunk, 629, 639, PS flippase, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '4', 'chunk': '14', 'confidence': '0.4721'}),
 Annotation(chunk, 641, 646, ATP8A1, {'entity': 'GENE', 'sentence': '4', 'chunk': '15', 'confidence': '0.9311'}),
 Annotation(chunk, 649, 657, ATPase II, {'entity': 'GENE', 'sentence': '4', 'chunk': '16', 'confidence': '0.7786'}),
 Annotation(chunk, 815, 832, yeast protein Drs2, {'entity': 'GENE', 'sentence': '4', 'chunk': '17', 'confidence': '0.6886'}),
 Annotation(chunk, 973, 978, ATP8A1, {'entity': 'GENE', 'sentence': '5', 'chunk': '18', 'confidence': '0.9956'}),
 Annotation(chunk, 1030, 1040, PS flippase, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '5', 'chunk': '19', 'confidence': '0.72085'}),
 Annotation(chunk, 1134, 1139, Atp8a1, {'entity': 'GENE', 'sentence': '6', 'chunk': '20', 'confidence': '0.9403'}),
 Annotation(chunk, 1201, 1219, phosphatidylcholine, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '21', 'confidence': '0.7827'}),
 Annotation(chunk, 1222, 1238, phosphatidic acid, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '22', 'confidence': '0.89865'}),
 Annotation(chunk, 1244, 1263, phosphatidylinositol, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '23', 'confidence': '0.9433'}),
 Annotation(chunk, 1292, 1311, phosphatidylglycerol, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '24', 'confidence': '0.7415'}),
 Annotation(chunk, 1316, 1339, phosphatidylethanolamine, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '25', 'confidence': '0.9951'}),
 Annotation(chunk, 1342, 1343, PE, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '26', 'confidence': '0.997'}),
 Annotation(chunk, 1377, 1378, PS, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '27', 'confidence': '0.9994'}),
 Annotation(chunk, 1401, 1402, PS, {'entity': 'CHEMICAL', 'sentence': '7', 'chunk': '28', 'confidence': '0.9989'}),
 Annotation(chunk, 1495, 1508, PS transporter, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '8', 'chunk': '29', 'confidence': '0.66905'}),
 Annotation(chunk, 1511, 1516, Atp8a1, {'entity': 'GENE', 'sentence': '8', 'chunk': '30', 'confidence': '0.996'}),
 Annotation(chunk, 1563, 1577, sn-1,2-glycerol, {'entity': 'CHEMICAL', 'sentence': '8', 'chunk': '31', 'confidence': '0.6535'}),
 Annotation(chunk, 1589, 159

In [0]:
results[0]["relations"]

Out[46]: [Annotation(category, 114, 248, ACTIVATOR, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.99321073', 'entity2_end': '248', 'chunk1': 'phosphatidylserine', 'entity2_begin': '224', 'entity1': 'CHEMICAL', 'entity1_begin': '114', 'entity1_end': '131', 'entity2': 'GENE_AND_CHEMICAL'}),
 Annotation(category, 134, 248, ACTIVATOR, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.6814849', 'entity2_end': '248', 'chunk1': 'PS', 'entity2_begin': '224', 'entity1': 'CHEMICAL', 'entity1_begin': '134', 'entity1_end': '135', 'entity2': 'GENE_AND_CHEMICAL'}),
 Annotation(category, 256, 309, SUBSTRATE, {'chunk2': 'PS', 'confidence': '0.999874', 'entity2_end': '309', 'chunk1': 'aminophospholipid "flippase', 'entity2_begin': '308', 'entity1': 'GENE_AND_CHEMICAL', 'entity1_begin': '256', 'entity1_end': '282', 'entity2': 'CHEMICAL'}),
 Annotation(category, 629, 646, PRODUCT-OF, {'chunk2': 'ATP8A1', 'confidence': '0.8554124', 'entity2_end': '646', 'chunk1': 'PS flippase', 'entity2_begin': '641', 'entity1': 'GENE_AND_CHEMICAL', 'entity1_begin': '629', 'entity1_end': '639', 'entity2': 'GENE'})]

In [0]:
rel_df = get_relations_df (results)

rel_df

Out[47]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 ACTIVATOR 
 CHEMICAL 
 114 
 131 
 phosphatidylserine 
 GENE_AND_CHEMICAL 
 224 
 248 
 ATP-dependent transporter 
 0.99321073 
 
 
 1 
 ACTIVATOR 
 CHEMICAL 
 134 
 135 
 PS 
 GENE_AND_CHEMICAL 
 224 
 248 
 ATP-dependent transporter 
 0.6814849 
 
 
 2 
 SUBSTRATE 
 GENE_AND_CHEMICAL 
 256 
 282 
 aminophospholipid "flippase 
 CHEMICAL 
 308 
 309 
 PS 
 0.999874 
 
 
 3 
 PRODUCT-OF 
 GENE_AND_CHEMICAL 
 629 
 639 
 PS flippase 
 GENE 
 641 
 646 
 ATP8A1 
 0.8554124

## Chemical–Protein Interactions (ChemProt RE)

Accurately detecting the interactions between chemicals and proteins is a crucial task that plays a key role in precision medicine, drug discovery and basic clinical research. Currently, PubMed contains >28 million articles, and its annual growth rate is more than a million articles each year. A large amount of valuable chemical–protein interactions (CPIs) are hidden in the biomedical literature. There is an increasing interest in CPI extraction from the biomedical literature.

Since manually extracting biomedical relations such as protein–protein interactions (PPI) and drug–drug interactions (DDI) is costly and time-consuming, some computational methods have been successfully proposed for automatic biomedical relation extraction.

To date, most studies on the biomedical relation extraction have focused on the PPIs and DDIs, but a few attempts have been made to extract CPIs. The BioCreative VI ChemProt shared task released the ChemProt dataset for CPI extraction, which is the first challenge for extracting CPIs.

Computational CPI extraction is generally approached as a task of classifying whether a specified semantic relation holds between the chemical and protein entities within a sentence or document. The ChemProt corpus is a manually annotated CPI dataset, which greatly promotes the development of CPI extraction approaches. 

ref: https://academic.oup.com/database/article/doi/10.1093/database/baz054/5498050

| Relation | Recall | Precision | F1 | F1 (Zhang, Yijia, et al., 2019) |
| --- | --- | --- | --- | --- |
| CPR:3 | 0.47 | 0.59 | **0.52** | 0.594 |
| CPR:4 | 0.72 | 0.81 | **0.77** | 0.718 |
| CPR:5 | 0.43 | 0.88 | **0.58** | 0.657 |
| CPR:6 | 0.59 | 0.89 | **0.71** | 0.725 |
| CPR:9 | 0.62 | 0.84 | **0.71** | 0.501 |
|avg. |  |  | **0.66** | 0.64 |

Here are the relation types

| Group  	| Eval. 	| CHEMPROT relations belonging to this group        	|
|--------	|-------	|---------------------------------------------------	|
| CPR:1  	| N     	| PART_OF                                           	|
| CPR:2  	| N     	| REGULATOR\|DIRECT REGULATOR\|INDIRECT REGULATOR   	|
| CPR:3  	| Y     	| UPREGULATOR ACTIVATOR INDIRECT UPREGULATOR        	|
| CPR:4  	| Y     	| DOWNREGULATORIINHIBITORIINDIRECTDOWNREGULATOR     	|
| CPR:5  	| Y     	| AGONISTIAGONIST-ACTIVATOR\|AGONIST-INHIBITOR      	|
| CPR:6  	| Y     	| ANTAGONIST                                        	|
| CPR:7  	| N     	| MODULATORIMODULATOR-ACTIVATORIMODULATOR-INHIBITOR 	|
| CPR:8  	| N     	| COFACTOR                                          	|
| CPR:9  	| Y     	| SUBSTRATEIPRODUCT OF\|SUBSTRATE PRODUCT OF        	|
| CPR:10 	| N     	| NOT                                               	|

ChemProt RE works well with `ner_chemprot_clinical` find relationships between the following entities

`CHEMICAL`: Chemical entity mention type; 

`GENE-Y`: gene/protein mention type that can be normalized or associated to a biological database identifier; 

`GENE-N`: gene/protein mention type that cannot be normalized to a database identifier.

In [0]:
ner_tagger = MedicalNerModel()\
    .pretrained("ner_chemprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverterInternal() \
    .setInputCols(["sentences", "tokens", "ner_tags"]) \
    .setOutputCol("ner_chunk")

chemprot_re_model = RelationExtractionModel()\
    .pretrained("re_chemprot_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunk", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\

chemprot_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_converter,
    dependency_parser,
    chemprot_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

chemprot_model = chemprot_pipeline.fit(empty_data)

ner_chemprot_clinical download started this may take some time.
Approximate size to download 15.3 MB
[ | ][ / ][OK!]
re_chemprot_clinical download started this may take some time.
Approximate size to download 5.9 MB
[ | ][OK!]

In [0]:
text='''
In this study, we examined the effects of mitiglinide on various cloned K(ATP) channels (Kir6.2/SUR1, Kir6.2/SUR2A, and Kir6.2/SUR2B) reconstituted in COS-1 cells, and compared them to another meglitinide-related compound, nateglinide. Patch-clamp analysis using inside-out recording configuration showed that mitiglinide inhibits the Kir6.2/SUR1 channel currents in a dose-dependent manner (IC50 value, 100 nM) but does not significantly inhibit either Kir6.2/SUR2A or Kir6.2/SUR2B channel currents even at high doses (more than 10 microM). Nateglinide inhibits Kir6.2/SUR1 and Kir6.2/SUR2B channels at 100 nM, and inhibits Kir6.2/SUR2A channels at high concentrations (1 microM). Binding experiments on mitiglinide, nateglinide, and repaglinide to SUR1 expressed in COS-1 cells revealed that they inhibit the binding of [3H]glibenclamide to SUR1 (IC50 values: mitiglinide, 280 nM; nateglinide, 8 microM; repaglinide, 1.6 microM), suggesting that they all share a glibenclamide binding site. The insulin responses to glucose, mitiglinide, tolbutamide, and glibenclamide in MIN6 cells after chronic mitiglinide, nateglinide, or repaglinide treatment were comparable to those after chronic tolbutamide and glibenclamide treatment. These results indicate that, similar to the sulfonylureas, mitiglinide is highly specific to the Kir6.2/SUR1 complex, i.e., the pancreatic beta-cell K(ATP) channel, and suggest that mitiglinide may be a clinically useful anti-diabetic drug.
'''
lmodel = LightPipeline(chemprot_model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.entity1!=rel_df.entity2]

Out[49]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 CPR:2 
 CHEMICAL 
 43 
 53 
 mitiglinide 
 GENE-N 
 80 
 87 
 channels 
 0.6070414 
 
 
 1 
 CPR:2 
 GENE-N 
 80 
 87 
 channels 
 CHEMICAL 
 224 
 234 
 nateglinide 
 0.98933965 
 
 
 4 
 CPR:4 
 CHEMICAL 
 706 
 716 
 mitiglinide 
 GENE-Y 
 751 
 754 
 SUR1 
 0.7594982 
 
 
 6 
 CPR:2 
 CHEMICAL 
 719 
 729 
 nateglinide 
 GENE-Y 
 751 
 754 
 SUR1 
 0.9999553 
 
 
 7 
 CPR:2 
 CHEMICAL 
 736 
 746 
 repaglinide 
 GENE-Y 
 751 
 754 
 SUR1 
 0.9919803 
 
 
 8 
 CPR:2 
 CHEMICAL 
 823 
 839 
 [3H]glibenclamide 
 GENE-Y 
 844 
 847 
 SUR1 
 1.0 
 
 
 11 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1019 
 1025 
 glucose 
 0.59488183 
 
 
 12 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1028 
 1038 
 mitiglinide 
 0.5599123 
 
 
 13 
 CPR:3 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1041 
 1051 
 tolbutamide 
 0.98802537 
 
 
 14 
 CPR:3 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1058 
 1070 
 glibenclamide 
 0.98900056 
 
 
 15 
 CPR:3 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1100 
 1110 
 mitiglinide 
 0.9898141 
 
 
 34 
 CPR:4 
 CHEMICAL 
 1290 
 1300 
 mitiglinide 
 GENE-N 
 1387 
 1393 
 channel 
 0.9894153

# Train a Custom Relation Extraction Model

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

dbutils.fs.cp("file:/databricks/driver/i2b2_clinical_rel_dataset.csv", "dbfs:/")

Out[77]: True

In [0]:
data = spark.read.option("header","true").format("csv").load("/i2b2_clinical_rel_dataset.csv")

data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')

data.show(10)

# you only need these columns>> 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel'
# ('dataset' column is optional)

+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
 sentence|firstCharEnt1|firstCharEnt2|lastCharEnt1|lastCharEnt2| chunk1| chunk2| label1| label2| rel|dataset|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
VITAL SIGNS - Tem...| 49| 75| 64| 84| respiratory rate| saturation| test| test| O| test|
No lotions , crea...| 3| 34| 9| 42| lotions| incisions|treatment| problem|TrNAP| test|
Because of expect...| 11| 58| 54| 68|expected long ter...| a picc line|treatment|treatment| O| train|
She states this l...| 16| 82| 31| 92| light-headedness| diaphoresis| problem| problem| PIP| train|
Initial electroca...| 196| 145| 239| 176|an inferior and r...|1-mm st depressio...| problem| problem| PIP| test|
Abdominal ultraso...| 1| 54| 20| 71|abdominal ultrasound| gallbladder sludge| test| problem| TeRP| test|
At that time he w...| 99| 139| 133| 173|ir placement of a...|his abdominopelvi...|treatment| problem| TrAP| test|
He was transferre...| 143| 195| 154| 213| reintubation| worsening pneumonia|treatment| problem| TrAP| test|
5) Anemia - previ...| 12| 35| 23| 47| previous w/u| elev ferritin| test| problem| TeRP| test|
The patient &apos...| 161| 149| 179| 156| po pain medications| his pain|treatment| problem| TrAP| train|
+--------------------+-------------+-------------+------------+------------+--------------------+--------------------+---------+---------+-----+-------+
only showing top 10 rows

In [0]:
data.groupby('dataset').count().show()

+-------+-----+
dataset|count|
+-------+-----+
 train| 350|
 test| 650|
+-------+-----+

In [0]:
#Annotation structure
annotationType = T.StructType([
            T.StructField('annotatorType', T.StringType(), False),
            T.StructField('begin', T.IntegerType(), False),
            T.StructField('end', T.IntegerType(), False),
            T.StructField('result', T.StringType(), False),
            T.StructField('metadata', T.MapType(T.StringType(), T.StringType()), False),
            T.StructField('embeddings', T.ArrayType(T.FloatType()), False)
        ])

#UDF function to convert train data to names entitities

@F.udf(T.ArrayType(annotationType))
def createTrainAnnotations(begin1, end1, begin2, end2, chunk1, chunk2, label1, label2):
    
    entity1 = sparknlp.annotation.Annotation("chunk", begin1, end1, chunk1, {'entity': label1.upper(), 'sentence': '0'}, [])
    entity2 = sparknlp.annotation.Annotation("chunk", begin2, end2, chunk2, {'entity': label2.upper(), 'sentence': '0'}, [])    
        
    entity1.annotatorType = "chunk"
    entity2.annotatorType = "chunk"

    return [entity1, entity2]    

#list of valid relations
rels = ["TrIP", "TrAP", "TeCP", "TrNAP", "TrCP", "PIP", "TrWP", "TeRP"]

#a query to select list of valid relations
valid_rel_query = "(" + " OR ".join(["rel = '{}'".format(rel) for rel in rels]) + ")"

data = data\
  .withColumn("begin1i", F.expr("cast(firstCharEnt1 AS Int)"))\
  .withColumn("end1i", F.expr("cast(lastCharEnt1 AS Int)"))\
  .withColumn("begin2i", F.expr("cast(firstCharEnt2 AS Int)"))\
  .withColumn("end2i", F.expr("cast(lastCharEnt2 AS Int)"))\
  .where("begin1i IS NOT NULL")\
  .where("end1i IS NOT NULL")\
  .where("begin2i IS NOT NULL")\
  .where("end2i IS NOT NULL")\
  .where(valid_rel_query)\
  .withColumn(
      "train_ner_chunks", 
      createTrainAnnotations(
          "begin1i", "end1i", "begin2i", "end2i", "chunk1", "chunk2", "label1", "label2"
      ).alias("train_ner_chunks", metadata={'annotatorType': "chunk"}))
    

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

In [0]:
#for training with existing graph 
"""!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/RE_in1200D_out20.pb -P /databricks/driver/re_graph/"""

In [0]:
%fs mkdirs file:/dbfs/re_graphs

res8: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/re_logs

res9: Boolean = true

In [0]:
%fs ls file:/dbfs

path name size file:/dbfs/ADE-NEG.txt ADE-NEG.txt 2308469 file:/dbfs/AntBNC_lemmas_ver_001.txt AntBNC_lemmas_ver_001.txt 1348552 file:/dbfs/AskAPatient.fold-0.test.txt AskAPatient.fold-0.test.txt 33834 file:/dbfs/AskAPatient.fold-0.train.txt AskAPatient.fold-0.train.txt 594794 file:/dbfs/AskAPatient.fold-0.validation.txt AskAPatient.fold-0.validation.txt 32162 file:/dbfs/DRUG-AE.rel DRUG-AE.rel 1425758 file:/dbfs/FileStore/ FileStore/ 4096 file:/dbfs/JohnSnowLabs/ JohnSnowLabs/ 4096 file:/dbfs/MT_00.pdf MT_00.pdf 528022 file:/dbfs/MT_01.pdf MT_01.pdf 165397 file:/dbfs/MT_02.pdf MT_02.pdf 309334 file:/dbfs/MT_03.pdf MT_03.pdf 248334 file:/dbfs/NCBI_disease_official_test.conll NCBI_disease_official_test.conll 328172 file:/dbfs/NCBI_disease_official_test.conll.1 NCBI_disease_official_test.conll.1 328172 file:/dbfs/NCBI_disease_official_test.conll.2 NCBI_disease_official_test.conll.2 328172 file:/dbfs/NCBI_disease_official_train_dev.conll NCBI_disease_official_train_dev.conll 2136193 file:/dbfs/NCBI_disease_official_train_dev.conll.1 NCBI_disease_official_train_dev.conll.1 2136193 file:/dbfs/NCBI_disease_official_train_dev.conll.2 NCBI_disease_official_train_dev.conll.2 2136193 file:/dbfs/NER_NCBIconlltest.parquet/ NER_NCBIconlltest.parquet/ 4096 file:/dbfs/NER_NCBIconlltest.txt NER_NCBIconlltest.txt 244358 file:/dbfs/NER_NCBIconlltrain.txt NER_NCBIconlltrain.txt 1091488 file:/dbfs/Users/ Users/ 4096 file:/dbfs/Workspace/ Workspace/ 4096 file:/dbfs/aclimdb_test.csv aclimdb_test.csv 32715164 file:/dbfs/aclimdb_train.csv aclimdb_train.csv 33497180 file:/dbfs/ade_data/ ade_data/ 4096 file:/dbfs/age.json age.json 434 file:/dbfs/amazon_alexa.tsv amazon_alexa.tsv 514752 file:/dbfs/amazon_alexa.tsv.1 amazon_alexa.tsv.1 514752 file:/dbfs/annotators.csv annotators.csv 3136 file:/dbfs/assertion_logs/ assertion_logs/ 4096 file:/dbfs/assertion_tf_graphs/ assertion_tf_graphs/ 4096 file:/dbfs/benchmark/ benchmark/ 4096 file:/dbfs/cities.json cities.json 101 file:/dbfs/cities.tsv cities.tsv 48 file:/dbfs/classifierdl_tensorflow classifierdl_tensorflow 22419981 file:/dbfs/classifierdl_transform/ classifierdl_transform/ 4096 file:/dbfs/classifierdl_transform_new/ classifierdl_transform_new/ 4096 file:/dbfs/clf_bert_logs/ clf_bert_logs/ 4096 file:/dbfs/clf_dl_logs/ clf_dl_logs/ 4096 file:/dbfs/clf_glove_logs/ clf_glove_logs/ 4096 file:/dbfs/clf_use_logs/ clf_use_logs/ 4096 file:/dbfs/custom_deid_masker_model/ custom_deid_masker_model/ 4096 file:/dbfs/custom_obfuscator_model/ custom_obfuscator_model/ 4096 file:/dbfs/databricks/ databricks/ 4096 file:/dbfs/databricks-datasets/ databricks-datasets/ 4096 file:/dbfs/databricks-results/ databricks-results/ 4096 file:/dbfs/datasets datasets 24032125 file:/dbfs/date.json date.json 205 file:/dbfs/dbfs/ dbfs/ 4096 file:/dbfs/deneme/ deneme/ 4096 file:/dbfs/dicom_1.dcm dicom_1.dcm 3836670 file:/dbfs/dicom_2.dcm dicom_2.dcm 640512 file:/dbfs/dicom_3.dcm dicom_3.dcm 900810 file:/dbfs/fields/ fields/ 4096 file:/dbfs/financial_entities.txt financial_entities.txt 27 file:/dbfs/gc_graph/ gc_graph/ 4096 file:/dbfs/gender.csv gender.csv 97 file:/dbfs/gender.json gender.json 75 file:/dbfs/generic_logs/ generic_logs/ 4096 file:/dbfs/hipaa-table-001.txt hipaa-table-001.txt 2497 file:/dbfs/hls hls 103558 file:/dbfs/holmes.txt holmes.txt 559722 file:/dbfs/home/ home/ 4096 file:/dbfs/i2b2_assertion_sample.csv i2b2_assertion_sample.csv 141585 file:/dbfs/i2b2_assertion_sample.csv.1 i2b2_assertion_sample.csv.1 141585 file:/dbfs/i2b2_assertion_sample.csv.2 i2b2_assertion_sample.csv.2 141585 file:/dbfs/i2b2_assertion_sample.csv.3 i2b2_assertion_sample.csv.3 141585 file:/dbfs/i2b2_assertion_sample.csv.4 i2b2_assertion_sample.csv.4 141585 file:/dbfs/i2b2_assertion_sample.csv.5 i2b2_assertion_sample.csv.5 141585 file:/dbfs/i2b2_assertion_sample.csv.6 i2b2_assertion_sample.csv.6 141585 file:/dbfs/i2b2_assertion_sample.csv.7 i2b2_assertion_sample.csv.7 141585 file:/dbfs/i2b2_assertion_sample.csv.8 i2b2_assertion_sample.csv.8 141585 

Graph and log folder has been created. 
Now, we will build graph and train the model <br/>
**WARNING:** For training a Relation Extraction model with custom graph, please use TensorFlow version 2.3

In [0]:
import tensorflow
from sparknlp_jsl.training import tf_graph

In [0]:
tf_graph.build("relation_extraction",build_params={"input_dim": 1200, "output_dim": 20, 'batch_norm':1, "hidden_layers": [300, 200], "hidden_act": "relu", 'hidden_act_l2':1}, model_location="/dbfs/re_graphs", model_filename="auto")

relation_extraction graph exported to /dbfs/re_graphs/rel_e.in1200.out20

In [0]:
%fs ls file:/dbfs/re_graphs

path name size file:/dbfs/re_graphs/rel_e.in1200.out20 rel_e.in1200.out20 109257

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")
    
dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

# set training params and upload model graph (see ../Healthcare/8.Generic_Classifier.ipynb)
reApproach = RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(50)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setModelFile("/dbfs/re_graphs/rel_e.in1200.out20")\
    .setFixImbalance(True)\
    .setFromEntity("begin1i", "end1i", "label1")\
    .setToEntity("begin2i", "end2i", "label2")\
    .setOutputLogsPath('dbfs:/re_logs')

finisher = Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

train_pipeline = Pipeline(stages=[
    documenter, tokenizer, words_embedder, pos_tagger, 
    dependency_parser, reApproach, finisher
])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]

In [0]:
rel_model = train_pipeline.fit(train_data)

In [0]:
rel_model.stages

Out[100]: [DocumentAssembler_987fe7e4cc16,
 REGEX_TOKENIZER_ab8e3dd41d62,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 dependency_d5a8da6c9093,
 RelationExtractionModel_e9a21e69c22e,
 Finisher_92f266eff29c]

In [0]:
rel_model.stages[-2]

Out[101]: RelationExtractionModel_e9a21e69c22e

In [0]:
rel_model.stages[-2].write().overwrite().save('dbfs:/databricks/driver/models/custom_RE_model')

In [0]:
result = rel_model.transform(test_data)

In [0]:
recall = result\
  .groupBy("rel")\
  .agg(F.avg(F.expr("IF(rel = relations_out, 1, 0)")).alias("recall"))\
  .select(
      F.col("rel").alias("relation"), 
      F.format_number("recall", 2).alias("recall"))\
  .show()

performance  = result\
  .where("relations_out <> ''")\
  .groupBy("relations_out")\
  .agg(F.avg(F.expr("IF(rel = relations_out, 1, 0)")).alias("precision"))\
  .select(
      F.col("relations_out").alias("relation"), 
      F.format_number("precision", 2).alias("precision"))\
  .show()

+--------+------+
relation|recall|
+--------+------+
 TrIP| 0.00|
 TrAP| 0.31|
 TeCP| 0.20|
 TrNAP| 0.00|
 TrCP| 0.00|
 PIP| 0.26|
 TrWP| 0.00|
 TeRP| 0.20|
+--------+------+

+--------+---------+
relation|precision|
+--------+---------+
 TrIP| 0.00|
 TrAP| 0.67|
 TeCP| 0.18|
 TrNAP| 0.00|
 TrCP| 0.00|
 PIP| 0.88|
 TeRP| 0.70|
+--------+---------+

In [0]:
result_df = result.select(F.explode(F.arrays_zip(result.relations.result, result.relations.metadata)).alias("cols")) \
.select(F.expr("cols['0']").alias("relation"),
        F.expr("cols['1']['entity1']").alias("entity1"),
        F.expr("cols['1']['entity1_begin']").alias("entity1_begin"),
        F.expr("cols['1']['entity1_end']").alias("entity1_end"),
        F.expr("cols['1']['chunk1']").alias("chunk1"),
        F.expr("cols['1']['entity2']").alias("entity2"),
        F.expr("cols['1']['entity2_begin']").alias("entity2_begin"),
        F.expr("cols['1']['entity2_end']").alias("entity2_end"),
        F.expr("cols['1']['chunk2']").alias("chunk2"),
        F.expr("cols['1']['confidence']").alias("confidence")
        )

result_df.show(50, truncate=100)

+--------+---------+-------------+-----------+----------------------------------------------------------+---------+-------------+-----------+-----------------------------------------------------------+----------+
relation| entity1|entity1_begin|entity1_end| chunk1| entity2|entity2_begin|entity2_end| chunk2|confidence|
+--------+---------+-------------+-----------+----------------------------------------------------------+---------+-------------+-----------+-----------------------------------------------------------+----------+
 PIP| PROBLEM| 145| 176| 1-mm st depressions in i and avl| PROBLEM| 196| 239| an inferior and right ventricular infarction| 0.7315465|
 TeCP| TEST| 1| 20| abdominal ultrasound| PROBLEM| 54| 71| gallbladder sludge| 0.7328917|
 TrAP|TREATMENT| 99| 133| ir placement of a drainage catheter| PROBLEM| 139| 173| his abdominopelvic fluid collection| 0.9134573|
 TeRP| TEST| 1| 8| a ct abd| PROBLEM| 16| 36| 8mm obstructing stone| 0.8411025|
 TrIP| PROBLEM| 111| 122| her symptoms|TREATMENT| 198| 209| levofloxacin| 0.5009705|
 TeRP| TEST| 1| 9| pathology| PROBLEM| 19| 77|poorly differentiated squamous cell carcinoma of the cervix|0.71671313|
 TeRP| PROBLEM| 1| 13| a small polyp|TREATMENT| 71| 81| an endoclip| 0.5042149|
 TeCP| TEST| 22| 44| cardiac catheterization| PROBLEM| 75| 103| a 20% left vein osteal lesion| 0.5775571|
 TrAP|TREATMENT| 10| 15| repair| PROBLEM| 20| 32| neck fracture|0.70843905|
 TrAP| PROBLEM| 3| 22| failed swallow study|TREATMENT| 36| 85| percutaneous endoscopic gastrostomy tube placement|0.57310176|
 TrAP|TREATMENT| 40| 85| postradical cystoprostatectomy with ileal loop| PROBLEM| 91| 122| locally invasive prostate cancer|0.88835967|
 TrAP|TREATMENT| 90| 99| management| PROBLEM| 104| 129| her pulmonary hypertension| 0.7892769|
 PIP| PROBLEM| 14| 48| a new two to three-pillow orthopnea| PROBLEM| 54| 81| paroxysmal nocturnal dyspnea| 0.6119761|
 TrAP| PROBLEM| 69| 80| pancytopenia|TREATMENT| 98| 100| ddi| 0.5085851|
 TrAP|TREATMENT| 14| 20| removal| PROBLEM| 25| 33| cbd stone| 0.6377525|
 TeRP| PROBLEM| 166| 223|posterior cerebral artery distribution cerebral infarction| PROBLEM| 230| 240| mass effect| 0.6216069|
 TrNAP| TEST| 10| 18| the study| PROBLEM| 112| 132| a cecal cystic lesion|0.65608615|
 TeCP| TEST| 8| 32| a cardiac catheterization| PROBLEM| 49| 64| 3-vessel disease|0.51944536|
 TrAP| PROBLEM| 309| 336| any bacterial superinfection| PROBLEM| 341| 362| acanthamoeba sinusitis| 0.8889965|
 TrAP|TREATMENT| 45| 54| vancomycin| PROBLEM| 149| 190| enteric and non-enteric gram negative rods|0.53652054|
 TrNAP| TEST| 17| 31| mediastinoscopy| PROBLEM| 185| 195| anthracosis| 0.9430044|
 PIP| PROBLEM| 55| 96| an additional dysfunction on the left side| PROBLEM| 109| 140| wide-spread cortical dysfunction| 0.7467205|
 TrNAP| TEST| 1| 3| ekg| PROBLEM| 86| 109| early r wave progression| 0.6642663|
 PIP| PROBLEM| 93| 112| q waves in leads iii| PROBLEM| 138| 171| anteroseptal myocardial infarction|0.66225296|
 TrNAP| TEST| 1| 3| ekg| PROBLEM| 80| 97| very small p waves| 0.7198931|
 TeRP| TEST| 1| 4| echo| PROBLEM| 48| 49| mr| 0.6844496|
 TeRP| PROBLEM| 45| 84| erosion of the gastroesophageal junction| PROBLEM| 182| 190| gastritis| 0.7761904|
 TeCP| TEST| 1| 17| an echocardiogram| PROBLEM| 35| 58| a dilated left ventricle| 0.7275105|
 TrAP| PROBLEM| 34| 48| his throat pain|TREATMENT| 74| 84| clindamycin|0.79235774|
 TrNAP| PROBLEM| 1| 36| his gastrointestinal bleeding issues| PROBLEM| 151| 202| an non-steroidal anti-inflammatory drugs gastropathy| 0.5053052|
 TeRP| TEST| 1| 6| an ekg| PROBLEM| 14| 34| st segment depression|0.78826207|
 TrAP|TREATMENT| 56| 65| management| PROBLEM| 70| 91| the increased bleeding| 0.791823|
 TeCP| PROBLEM| 41| 48| elevated| TEST| 75| 89| further work-up|0.61624086|
 TrIP| PROBLEM| 1| 15| his throat pain|TREATMENT| 78| 86| ibuprofen| 0.8466852|
 TeCP| TEST| 24| 37| chest ct scans| PROBLEM| 64| 78| pulmonary edema| 0.5559204|
 TeCP| TEST| 1| 34| subsequent cardiac catheteriza

In [0]:
result_df.limit(20).toPandas()

Out[106]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 PIP 
 PROBLEM 
 145 
 176 
 1-mm st depressions in i and avl 
 PROBLEM 
 196 
 239 
 an inferior and right ventricular infarction 
 0.7315465 
 
 
 1 
 TeCP 
 TEST 
 1 
 20 
 abdominal ultrasound 
 PROBLEM 
 54 
 71 
 gallbladder sludge 
 0.7328917 
 
 
 2 
 TrAP 
 TREATMENT 
 99 
 133 
 ir placement of a drainage catheter 
 PROBLEM 
 139 
 173 
 his abdominopelvic fluid collection 
 0.9134573 
 
 
 3 
 TeRP 
 TEST 
 1 
 8 
 a ct abd 
 PROBLEM 
 16 
 36 
 8mm obstructing stone 
 0.8411025 
 
 
 4 
 TrIP 
 PROBLEM 
 111 
 122 
 her symptoms 
 TREATMENT 
 198 
 209 
 levofloxacin 
 0.5009705 
 
 
 5 
 TeRP 
 TEST 
 1 
 9 
 pathology 
 PROBLEM 
 19 
 77 
 poorly differentiated squamous cell carcinoma of the cervix 
 0.71671313 
 
 
 6 
 TeRP 
 PROBLEM 
 1 
 13 
 a small polyp 
 TREATMENT 
 71 
 81 
 an endoclip 
 0.5042149 
 
 
 7 
 TeCP 
 TEST 
 22 
 44 
 cardiac catheterization 
 PROBLEM 
 75 
 103 
 a 20% left vein osteal lesion 
 0.5775571 
 
 
 8 
 TrAP 
 TREATMENT 
 10 
 15 
 repair 
 PROBLEM 
 20 
 32 
 neck fracture 
 0.70843905 
 
 
 9 
 TrAP 
 PROBLEM 
 3 
 22 
 failed swallow study 
 TREATMENT 
 36 
 85 
 percutaneous endoscopic gastrostomy tube placement 
 0.57310176 
 
 
 10 
 TrAP 
 TREATMENT 
 40 
 85 
 postradical cystoprostatectomy with ileal loop 
 PROBLEM 
 91 
 122 
 locally invasive prostate cancer 
 0.88835967 
 
 
 11 
 TrAP 
 TREATMENT 
 90 
 99 
 management 
 PROBLEM 
 104 
 129 
 her pulmonary hypertension 
 0.7892769 
 
 
 12 
 PIP 
 PROBLEM 
 14 
 48 
 a new two to three-pillow orthopnea 
 PROBLEM 
 54 
 81 
 paroxysmal nocturnal dyspnea 
 0.6119761 
 
 
 13 
 TrAP 
 PROBLEM 
 69 
 80 
 pancytopenia 
 TREATMENT 
 98 
 100 
 ddi 
 0.5085851 
 
 
 14 
 TrAP 
 TREATMENT 
 14 
 20 
 removal 
 PROBLEM 
 25 
 33 
 cbd stone 
 0.6377525 
 
 
 15 
 TeRP 
 PROBLEM 
 166 
 223 
 posterior cerebral artery distribution cerebral infarction 
 PROBLEM 
 230 
 240 
 mass effect 
 0.6216069 
 
 
 16 
 TrNAP 
 TEST 
 10 
 18 
 the study 
 PROBLEM 
 112 
 132 
 a cecal cystic lesion 
 0.65608615 
 
 
 17 
 TeCP 
 TEST 
 8 
 32 
 a cardiac catheterization 
 PROBLEM 
 49 
 64 
 3-vessel disease 
 0.51944536 
 
 
 18 
 TrAP 
 PROBLEM 
 309 
 336 
 any bacterial superinfection 
 PROBLEM 
 341 
 362 
 acanthamoeba sinusitis 
 0.8889965 
 
 
 19 
 TrAP 
 TREATMENT 
 45 
 54 
 vancomycin 
 PROBLEM 
 149 
 190 
 enteric and non-enteric gram negative rods 
 0.53652054

## Load trained model from disk

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

clinical_ner_tagger = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "tokens", "embeddings")\
    .setOutputCol("ner_tags") 

loaded_re_Model = RelationExtractionModel.load("dbfs:/databricks/driver/models/custom_RE_model")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"]) \
    .setOutputCol("relations")\
    .setRelationPairs(["problem-test", "problem-treatment"])\
    .setPredictionThreshold(0.7)\
    .setMaxSyntacticDistance(6)

trained_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    loaded_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

loaded_re_model = trained_pipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]

In [0]:

text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge.
"""

loaded_re_model_light = LightPipeline(loaded_re_model)
annotations = loaded_re_model_light.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.relation!="O"]

Out[112]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 TrNAP 
 PROBLEM 
 157 
 160 
 T2DM 
 TEST 
 322 
 324 
 BMI 
 0.990436 
 
 
 1 
 TrAP 
 TREATMENT 
 512 
 522 
 amoxicillin 
 PROBLEM 
 528 
 556 
 a respiratory tract infection 
 0.8473756 
 
 
 2 
 TrAP 
 TREATMENT 
 599 
 611 
 dapagliflozin 
 PROBLEM 
 617 
 620 
 T2DM 
 0.75986147 
 
 
 3 
 TrAP 
 TREATMENT 
 643 
 653 
 gemfibrozil 
 PROBLEM 
 659 
 661 
 HTG 
 0.7700838

# End-to-end trained Models using BioBERT
#### Latest addition to Spark NLP for Heathcare - (Requires Spark NLP 2.7.3+ and Spark NLP JSL 2.7.3+)

These models are trained as end-to-end bert models using BioBERT and ported in to the Spark NLP ecosystem. 

They offer SOTA performance on most benchmark tasks and outperform our existing Relation Extraction Models.

## ADE ReDL

In [0]:
ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")  

ade_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

ade_re_model = RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

ade_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_model = ade_pipeline.fit(empty_data)

ner_ade_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
redl_ade_biobert download started this may take some time.
Approximate size to download 383.3 MB
[ | ][OK!]

In [0]:
text ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

ade_lmodel = LightPipeline(ade_model)
ade_results = ade_lmodel.fullAnnotate(text)

rel_df = get_relations_df (ade_results)

rel_df

Out[54]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 1 
 DRUG 
 25 
 32 
 naproxen 
 ADE 
 137 
 148 
 tense bullae 
 0.9993493 
 
 
 1 
 1 
 DRUG 
 25 
 32 
 naproxen 
 ADE 
 154 
 210 
 cutaneous fragility on the face and the back of the hands 
 0.99964464 
 
 
 2 
 1 
 DRUG 
 87 
 95 
 oxaprozin 
 ADE 
 137 
 148 
 tense bullae 
 0.9990792 
 
 
 3 
 1 
 DRUG 
 87 
 95 
 oxaprozin 
 ADE 
 154 
 210 
 cutaneous fragility on the face and the back of the hands 
 0.998869

## Clinical ReDL

In [0]:
clinical_ner_tagger = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)\
    .setRelationPairs(["problem-test", "problem-treatment"])# we can set the possible relation pairs (if not set, all the relations will be calculated)
    
clinical_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_clinical_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_ner_chunk_filter,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
redl_clinical_biobert download started this may take some time.
Approximate size to download 383.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
text ="""A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), 
one prior episode of HTG-induced pancreatitis three years prior to presentation,  associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . 
She had close follow-up with endocrinology post discharge .
"""
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df

Out[56]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 TrAP 
 TREATMENT 
 512 
 522 
 amoxicillin 
 PROBLEM 
 528 
 556 
 a respiratory tract infection 
 0.997001 
 
 
 1 
 TrAP 
 TREATMENT 
 571 
 579 
 metformin 
 PROBLEM 
 617 
 620 
 T2DM 
 0.9962547 
 
 
 2 
 TrAP 
 TREATMENT 
 583 
 591 
 glipizide 
 PROBLEM 
 617 
 620 
 T2DM 
 0.9948763 
 
 
 3 
 TrAP 
 TREATMENT 
 599 
 611 
 dapagliflozin 
 PROBLEM 
 617 
 620 
 T2DM 
 0.9976178 
 
 
 4 
 TrAP 
 TREATMENT 
 599 
 611 
 dapagliflozin 
 PROBLEM 
 659 
 661 
 HTG 
 0.9978381 
 
 
 5 
 TrAP 
 PROBLEM 
 617 
 620 
 T2DM 
 TREATMENT 
 626 
 637 
 atorvastatin 
 0.9946819 
 
 
 6 
 TrAP 
 PROBLEM 
 617 
 620 
 T2DM 
 TREATMENT 
 643 
 653 
 gemfibrozil 
 0.9951603 
 
 
 7 
 TrAP 
 TREATMENT 
 626 
 637 
 atorvastatin 
 PROBLEM 
 659 
 661 
 HTG 
 0.96257573 
 
 
 8 
 TrAP 
 TREATMENT 
 643 
 653 
 gemfibrozil 
 PROBLEM 
 659 
 661 
 HTG 
 0.9920128 
 
 
 9 
 TeRP 
 TEST 
 739 
 758 
 Physical examination 
 PROBLEM 
 796 
 810 
 dry oral mucosa 
 0.99468684 
 
 
 10 
 TeRP 
 TEST 
 830 
 854 
 her abdominal examination 
 PROBLEM 
 875 
 884 
 tenderness 
 0.994568 
 
 
 11 
 TeRP 
 TEST 
 830 
 854 
 her abdominal examination 
 PROBLEM 
 888 
 895 
 guarding 
 0.9942286 
 
 
 12 
 TeRP 
 TEST 
 830 
 854 
 her abdominal examination 
 PROBLEM 
 902 
 909 
 rigidity 
 0.99472505 
 
 
 13 
 TeRP 
 TEST 
 1507 
 1517 
 her glucose 
 PROBLEM 
 1553 
 1566 
 still elevated 
 0.98072183 
 
 
 14 
 TeRP 
 TEST 
 1535 
 1547 
 the anion gap 
 PROBLEM 
 1553 
 1566 
 still elevated 
 0.98780394 
 
 
 15 
 TeRP 
 PROBLEM 
 1553 
 1566 
 still elevated 
 TEST 
 1576 
 1592 
 serum bicarbonate 
 0.9505035 
 
 
 16 
 TeRP 
 PROBLEM 
 1553 
 1566 
 still elevated 
 TEST 
 1670 
 1672 
 U/L 
 0.946596 
 
 
 17 
 TeRP 
 TEST 
 1676 
 1702 
 The β-hydroxybutyrate level 
 PROBLEM 
 1733 
 1740 
 elevated 
 0.98575115 
 
 
 18 
 TrAP 
 TREATMENT 
 1937 
 1951 
 an insulin drip 
 PROBLEM 
 1957 
 1961 
 euDKA 
 0.9941168

## Clinical Temporal Events ReDL

In [0]:
events_ner_tagger = MedicalNerModel.pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")     

events_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")
    
events_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_temporal_events_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    events_re_ner_chunk_filter,
    events_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
redl_temporal_events_biobert download started this may take some time.
Approximate size to download 383.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
text ="She is diagnosed as cancer in 1991. Then she was admitted to Mayo Clinic in May 2000 and discharged in October 2001"

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df = rel_df[(rel_df.relation!="O")]

rel_df[(rel_df.relation!="O")&(rel_df.entity1!=rel_df.entity2)]


Out[58]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 OVERLAP 
 PROBLEM 
 20 
 25 
 cancer 
 DATE 
 30 
 33 
 1991 
 0.754804 
 
 
 1 
 AFTER 
 OCCURRENCE 
 49 
 56 
 admitted 
 CLINICAL_DEPT 
 61 
 71 
 Mayo Clinic 
 0.849415 
 
 
 2 
 OVERLAP 
 OCCURRENCE 
 49 
 56 
 admitted 
 DATE 
 76 
 83 
 May 2000 
 0.599573 
 
 
 4 
 OVERLAP 
 CLINICAL_DEPT 
 61 
 71 
 Mayo Clinic 
 DATE 
 76 
 83 
 May 2000 
 0.811123 
 
 
 5 
 BEFORE 
 CLINICAL_DEPT 
 61 
 71 
 Mayo Clinic 
 OCCURRENCE 
 89 
 98 
 discharged 
 0.520918 
 
 
 6 
 OVERLAP 
 CLINICAL_DEPT 
 61 
 71 
 Mayo Clinic 
 DATE 
 103 
 114 
 October 2001 
 0.554499

In [0]:
text ="""She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed.  
She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 
12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. 
"""

annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df.confidence = rel_df.confidence.astype(float)

rel_df[(rel_df.relation!="O")]


Out[59]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 AFTER 
 OCCURRENCE 
 7 
 14 
 admitted 
 CLINICAL_DEPT 
 19 
 43 
 The John Hopkins Hospital 
 0.918168 
 
 
 1 
 BEFORE 
 OCCURRENCE 
 7 
 14 
 admitted 
 DATE 
 45 
 54 
 2 days ago 
 0.602319 
 
 
 2 
 BEFORE 
 OCCURRENCE 
 7 
 14 
 admitted 
 PROBLEM 
 74 
 102 
 gestational diabetes mellitus 
 0.863136 
 
 
 3 
 OVERLAP 
 CLINICAL_DEPT 
 19 
 43 
 The John Hopkins Hospital 
 DATE 
 45 
 54 
 2 days ago 
 0.722872 
 
 
 4 
 BEFORE 
 CLINICAL_DEPT 
 19 
 43 
 The John Hopkins Hospital 
 PROBLEM 
 74 
 102 
 gestational diabetes mellitus 
 0.519889 
 
 
 5 
 BEFORE 
 DATE 
 45 
 54 
 2 days ago 
 PROBLEM 
 74 
 102 
 gestational diabetes mellitus 
 0.507343 
 
 
 6 
 BEFORE 
 CLINICAL_DEPT 
 133 
 157 
 the endocrinology service 
 OCCURRENCE 
 171 
 180 
 discharged 
 0.575047 
 
 
 7 
 OVERLAP 
 CLINICAL_DEPT 
 133 
 157 
 the endocrinology service 
 DATE 
 185 
 194 
 03/02/2018 
 0.697982 
 
 
 8 
 OVERLAP 
 CLINICAL_DEPT 
 133 
 157 
 the endocrinology service 
 TREATMENT 
 211 
 226 
 insulin glargine 
 0.743572 
 
 
 9 
 OVERLAP 
 CLINICAL_DEPT 
 133 
 157 
 the endocrinology service 
 TREATMENT 
 242 
 255 
 insulin lispro 
 0.726185 
 
 
 10 
 OVERLAP 
 CLINICAL_DEPT 
 133 
 157 
 the endocrinology service 
 TREATMENT 
 262 
 270 
 metformin 
 0.930377 
 
 
 11 
 OVERLAP 
 CLINICAL_DEPT 
 133 
 157 
 the endocrinology service 
 FREQUENCY 
 280 
 294 
 two times a day 
 0.803400 
 
 
 12 
 OVERLAP 
 OCCURRENCE 
 171 
 180 
 discharged 
 DATE 
 185 
 194 
 03/02/2018 
 0.667649 
 
 
 13 
 BEFORE 
 OCCURRENCE 
 171 
 180 
 discharged 
 TREATMENT 
 211 
 226 
 insulin glargine 
 0.639843 
 
 
 14 
 BEFORE 
 OCCURRENCE 
 171 
 180 
 discharged 
 TREATMENT 
 242 
 255 
 insulin lispro 
 0.685371 
 
 
 15 
 BEFORE 
 OCCURRENCE 
 171 
 180 
 discharged 
 TREATMENT 
 262 
 270 
 metformin 
 0.690107 
 
 
 16 
 BEFORE 
 OCCURRENCE 
 171 
 180 
 discharged 
 FREQUENCY 
 280 
 294 
 two times a day 
 0.646173 
 
 
 17 
 OVERLAP 
 DATE 
 185 
 194 
 03/02/2018 
 TREATMENT 
 211 
 226 
 insulin glargine 
 0.875645 
 
 
 18 
 OVERLAP 
 DATE 
 185 
 194 
 03/02/2018 
 TREATMENT 
 242 
 255 
 insulin lispro 
 0.768634 
 
 
 19 
 OVERLAP 
 DATE 
 185 
 194 
 03/02/2018 
 TREATMENT 
 262 
 270 
 metformin 
 0.882646 
 
 
 20 
 OVERLAP 
 DATE 
 185 
 194 
 03/02/2018 
 FREQUENCY 
 280 
 294 
 two times a day 
 0.786517 
 
 
 21 
 OVERLAP 
 TREATMENT 
 211 
 226 
 insulin glargine 
 TREATMENT 
 242 
 255 
 insulin lispro 
 0.957287 
 
 
 22 
 OVERLAP 
 TREATMENT 
 211 
 226 
 insulin glargine 
 TREATMENT 
 262 
 270 
 metformin 
 0.992474 
 
 
 23 
 OVERLAP 
 TREATMENT 
 211 
 226 
 insulin glargine 
 FREQUENCY 
 280 
 294 
 two times a day 
 0.937546 
 
 
 24 
 OVERLAP 
 TREATMENT 
 242 
 255 
 insulin lispro 
 TREATMENT 
 262 
 270 
 metformin 
 0.987487 
 
 
 25 
 OVERLAP 
 TREATMENT 
 242 
 255 
 insulin lispro 
 FREQUENCY 
 280 
 294 
 two times a day 
 0.938404 
 
 
 26 
 OVERLAP 
 TREATMENT 
 262 
 270 
 metformin 
 FREQUENCY 
 280 
 294 
 two times a day 
 0.979315

## Human Phenotype - Gene ReDL

In [0]:
pgr_ner_tagger = MedicalNerModel.pretrained("ner_human_phenotype_gene_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pgr_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)
    
pgr_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_human_phenotype_gene_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pgr_ner_tagger,
    ner_chunker,
    dependency_parser,
    pgr_re_ner_chunk_filter,
    pgr_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_human_phenotype_gene_clinical download started this may take some time.
Approximate size to download 14 MB
[ | ][OK!]
redl_human_phenotype_gene_biobert download started this may take some time.
Approximate size to download 383.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
text = "She has a retinal degeneration, hearing loss and renal failure, short stature, \
Mutations in the SH3PXD2B gene coding for the Tks4 protein are responsible for the autosomal recessive."

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df = rel_df[(rel_df.relation!=0)]

rel_df

Out[61]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 0 
 HP 
 10 
 29 
 retinal degeneration 
 HP 
 32 
 43 
 hearing loss 
 0.945751 
 
 
 1 
 0 
 HP 
 10 
 29 
 retinal degeneration 
 HP 
 49 
 61 
 renal failure 
 0.9613808 
 
 
 2 
 1 
 HP 
 10 
 29 
 retinal degeneration 
 HP 
 162 
 180 
 autosomal recessive 
 0.5767865 
 
 
 3 
 0 
 HP 
 32 
 43 
 hearing loss 
 HP 
 49 
 61 
 renal failure 
 0.97126263 
 
 
 4 
 0 
 HP 
 32 
 43 
 hearing loss 
 HP 
 64 
 76 
 short stature 
 0.68042266 
 
 
 5 
 1 
 HP 
 32 
 43 
 hearing loss 
 GENE 
 96 
 103 
 SH3PXD2B 
 0.82296234 
 
 
 6 
 1 
 HP 
 32 
 43 
 hearing loss 
 HP 
 162 
 180 
 autosomal recessive 
 0.898228 
 
 
 7 
 0 
 HP 
 49 
 61 
 renal failure 
 HP 
 64 
 76 
 short stature 
 0.8272672 
 
 
 8 
 1 
 HP 
 49 
 61 
 renal failure 
 GENE 
 96 
 103 
 SH3PXD2B 
 0.6609864 
 
 
 9 
 1 
 HP 
 49 
 61 
 renal failure 
 HP 
 162 
 180 
 autosomal recessive 
 0.832035 
 
 
 10 
 1 
 HP 
 64 
 76 
 short stature 
 GENE 
 96 
 103 
 SH3PXD2B 
 0.8399175

## Drug-Drug Interaction ReDL

In [0]:
ddi_ner_tagger = MedicalNerModel.pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ddi_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)
    
ddi_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_drug_drug_interaction_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ddi_ner_tagger,
    ner_chunker,
    dependency_parser,
    ddi_re_ner_chunk_filter,
    ddi_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
redl_drug_drug_interaction_biobert download started this may take some time.
Approximate size to download 383.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
text='When carbamazepine is withdrawn from the combination therapy, aripiprazole dose should then be reduced. \
If additional adrenergic drugs are to be administered by any route, \
they should be used with caution because the pharmacologically predictable sympathetic effects of Metformin may be potentiated'

lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df

Out[63]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 DDI-false 
 DRUG 
 5 
 17 
 carbamazepine 
 DRUG 
 62 
 73 
 aripiprazole 
 0.9105875

## Drug-Protein Interaction ReDL

In [0]:
drugprot_ner_tagger = MedicalNerModel.pretrained("ner_drugprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

ner_converter = NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

# Set a filter on pairs of named entities which will be treated as relation candidates
drugprot_re_ner_chunk_filter = RENerChunksFilter()\
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)\
   .setRelationPairs(["checmical-gene", "chemical-gene_and_chemical", "gene_and_chemical-gene"])
    
drugprot_re_Model = RelationExtractionDLModel()\
    .pretrained('redl_drugprot_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[documenter, sentencer, tokenizer, words_embedder, drugprot_ner_tagger, ner_converter, pos_tagger, dependency_parser, drugprot_re_ner_chunk_filter, drugprot_re_Model])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_drugprot_clinical download started this may take some time.
Approximate size to download 14 MB
[ | ][OK!]
redl_drugprot_biobert download started this may take some time.
Approximate size to download 386.6 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
text='''Lipid specific activation of the murine P4-ATPase Atp8a1 (ATPase II). The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter. This aminophospholipid "flippase" selectively transports PS to the cytosolic leaflet of the bilayer and is sensitive to vanadate, Ca(2+), and modification by sulfhydryl reagents. Although the flippase has not been positively identified, a subfamily of P-type ATPases has been proposed to function as transporters of amphipaths, including PS and other phospholipids. A candidate PS flippase ATP8A1 (ATPase II), originally isolated from bovine secretory vesicles, is a member of this subfamily based on sequence homology to the founding member of the subfamily, the yeast protein Drs2, which has been linked to ribosomal assembly, the formation of Golgi-coated vesicles, and the maintenance of PS asymmetry. To determine if ATP8A1 has biochemical characteristics consistent with a PS flippase, a murine homologue of this enzyme was expressed in insect cells and purified. The purified Atp8a1 is inactive in detergent micelles or in micelles containing phosphatidylcholine, phosphatidic acid, or phosphatidylinositol, is minimally activated by phosphatidylglycerol or phosphatidylethanolamine (PE), and is maximally activated by PS. The selectivity for PS is dependent upon multiple elements of the lipid structure. Similar to the plasma membrane PS transporter, Atp8a1 is activated only by the naturally occurring sn-1,2-glycerol isomer of PS and not the sn-2,3-glycerol stereoisomer. Both flippase and Atp8a1 activities are insensitive to the stereochemistry of the serine headgroup. Most modifications of the PS headgroup structure decrease recognition by the plasma membrane PS flippase. Activation of Atp8a1 is also reduced by these modifications; phosphatidylserine-O-methyl ester, lysophosphatidylserine, glycerophosphoserine, and phosphoserine, which are not transported by the plasma membrane flippase, do not activate Atp8a1. Weakly translocated lipids (PE, phosphatidylhydroxypropionate, and phosphatidylhomoserine) are also weak Atp8a1 activators. However, N-methyl-phosphatidylserine, which is transported by the plasma membrane flippase at a rate equivalent to PS, is incapable of activating Atp8a1 activity. These results indicate that the ATPase activity of the secretory granule Atp8a1 is activated by phospholipids binding to a specific site whose properties (PS selectivity, dependence upon glycerol but not serine, stereochemistry, and vanadate sensitivity) are similar to, but distinct from, the properties of the substrate binding site of the plasma membrane flippase.'''

lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [0]:
results[0].keys()

Out[66]: dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 're_ner_chunks', 'dependencies'])

In [0]:
results[0]["ner_chunks"]

Out[67]: [Annotation(chunk, 33, 48, murine P4-ATPase, {'entity': 'GENE', 'sentence': '0', 'chunk': '0', 'confidence': '0.68700004'}),
 Annotation(chunk, 50, 55, Atp8a1, {'entity': 'GENE', 'sentence': '0', 'chunk': '1', 'confidence': '0.9957'}),
 Annotation(chunk, 58, 66, ATPase II, {'entity': 'GENE', 'sentence': '0', 'chunk': '2', 'confidence': '0.77655'}),
 Annotation(chunk, 114, 131, phosphatidylserine, {'entity': 'CHEMICAL', 'sentence': '1', 'chunk': '3', 'confidence': '0.7679'}),
 Annotation(chunk, 134, 135, PS, {'entity': 'CHEMICAL', 'sentence': '1', 'chunk': '4', 'confidence': '0.9767'}),
 Annotation(chunk, 224, 248, ATP-dependent transporter, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '1', 'chunk': '5', 'confidence': '0.87095'}),
 Annotation(chunk, 256, 282, aminophospholipid "flippase, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '2', 'chunk': '6', 'confidence': '0.5996'}),
 Annotation(chunk, 308, 309, PS, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '7', 'confidence': '0.9969'}),
 Annotation(chunk, 371, 378, vanadate, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '8', 'confidence': '0.9464'}),
 Annotation(chunk, 381, 385, Ca(2+, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '9', 'confidence': '0.5832'}),
 Annotation(chunk, 409, 418, sulfhydryl, {'entity': 'CHEMICAL', 'sentence': '2', 'chunk': '10', 'confidence': '0.7311'}),
 Annotation(chunk, 443, 450, flippase, {'entity': 'GENE', 'sentence': '3', 'chunk': '11', 'confidence': '0.9773'}),
 Annotation(chunk, 503, 516, P-type ATPases, {'entity': 'GENE', 'sentence': '3', 'chunk': '12', 'confidence': '0.73015'}),
 Annotation(chunk, 589, 590, PS, {'entity': 'CHEMICAL', 'sentence': '3', 'chunk': '13', 'confidence': '0.9953'}),
 Annotation(chunk, 629, 639, PS flippase, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '4', 'chunk': '14', 'confidence': '0.4721'}),
 Annotation(chunk, 641, 646, ATP8A1, {'entity': 'GENE', 'sentence': '4', 'chunk': '15', 'confidence': '0.9311'}),
 Annotation(chunk, 649, 657, ATPase II, {'entity': 'GENE', 'sentence': '4', 'chunk': '16', 'confidence': '0.7786'}),
 Annotation(chunk, 815, 832, yeast protein Drs2, {'entity': 'GENE', 'sentence': '4', 'chunk': '17', 'confidence': '0.6886'}),
 Annotation(chunk, 973, 978, ATP8A1, {'entity': 'GENE', 'sentence': '5', 'chunk': '18', 'confidence': '0.9956'}),
 Annotation(chunk, 1030, 1040, PS flippase, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '5', 'chunk': '19', 'confidence': '0.72085'}),
 Annotation(chunk, 1134, 1139, Atp8a1, {'entity': 'GENE', 'sentence': '6', 'chunk': '20', 'confidence': '0.9403'}),
 Annotation(chunk, 1201, 1219, phosphatidylcholine, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '21', 'confidence': '0.7827'}),
 Annotation(chunk, 1222, 1238, phosphatidic acid, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '22', 'confidence': '0.89865'}),
 Annotation(chunk, 1244, 1263, phosphatidylinositol, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '23', 'confidence': '0.9433'}),
 Annotation(chunk, 1292, 1311, phosphatidylglycerol, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '24', 'confidence': '0.7415'}),
 Annotation(chunk, 1316, 1339, phosphatidylethanolamine, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '25', 'confidence': '0.9951'}),
 Annotation(chunk, 1342, 1343, PE, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '26', 'confidence': '0.997'}),
 Annotation(chunk, 1377, 1378, PS, {'entity': 'CHEMICAL', 'sentence': '6', 'chunk': '27', 'confidence': '0.9994'}),
 Annotation(chunk, 1401, 1402, PS, {'entity': 'CHEMICAL', 'sentence': '7', 'chunk': '28', 'confidence': '0.9989'}),
 Annotation(chunk, 1495, 1508, PS transporter, {'entity': 'GENE_AND_CHEMICAL', 'sentence': '8', 'chunk': '29', 'confidence': '0.66905'}),
 Annotation(chunk, 1511, 1516, Atp8a1, {'entity': 'GENE', 'sentence': '8', 'chunk': '30', 'confidence': '0.996'}),
 Annotation(chunk, 1563, 1577, sn-1,2-glycerol, {'entity': 'CHEMICAL', 'sentence': '8', 'chunk': '31', 'confidence': '0.6535'}),
 Annotation(chunk, 1589, 159

In [0]:
results[0]["relations"]

Out[68]: [Annotation(category, 114, 248, SUBSTRATE, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.98467976', 'entity2_end': '248', 'syntactic_distance': '3', 'chunk1': 'phosphatidylserine', 'entity2_begin': '224', 'entity1': 'CHEMICAL', 'entity1_begin': '114', 'entity1_end': '131', 'context': 'The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter.', 'entity2': 'GENE_AND_CHEMICAL'}),
 Annotation(category, 134, 248, SUBSTRATE, {'chunk2': 'ATP-dependent transporter', 'confidence': '0.9953898', 'entity2_end': '248', 'syntactic_distance': '4', 'chunk1': 'PS', 'entity2_begin': '224', 'entity1': 'CHEMICAL', 'entity1_begin': '134', 'entity1_end': '135', 'context': 'The asymmetric transbilayer distribution of phosphatidylserine (PS) in the mammalian plasma membrane and secretory vesicles is maintained, in part, by an ATP-dependent transporter.', 'entity2': 'GENE_AND_CHEMICAL'}),
 Annotation(category, 256, 309, SUBSTRATE, {'chunk2': 'PS', 'confidence': '0.9982186', 'entity2_end': '309', 'syntactic_distance': '2', 'chunk1': 'aminophospholipid "flippase', 'entity2_begin': '308', 'entity1': 'GENE_AND_CHEMICAL', 'entity1_begin': '256', 'entity1_end': '282', 'context': 'This aminophospholipid "flippase" selectively transports PS to the cytosolic leaflet of the bilayer and is sensitive to vanadate, Ca(2+), and modification by sulfhydryl reagents.', 'entity2': 'CHEMICAL'})]

In [0]:
rel_df = get_relations_df (results)

rel_df

Out[69]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 SUBSTRATE 
 CHEMICAL 
 114 
 131 
 phosphatidylserine 
 GENE_AND_CHEMICAL 
 224 
 248 
 ATP-dependent transporter 
 0.98467976 
 
 
 1 
 SUBSTRATE 
 CHEMICAL 
 134 
 135 
 PS 
 GENE_AND_CHEMICAL 
 224 
 248 
 ATP-dependent transporter 
 0.9953898 
 
 
 2 
 SUBSTRATE 
 GENE_AND_CHEMICAL 
 256 
 282 
 aminophospholipid "flippase 
 CHEMICAL 
 308 
 309 
 PS 
 0.9982186

## Chemical–Protein Interactions (ChemProt) ReDL

In [0]:
chemprot_ner_tagger = MedicalNerModel.pretrained("ner_chemprot_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

chemprot_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)
    
chemprot_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_chemprot_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.9)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    chemprot_ner_tagger,
    ner_chunker,
    dependency_parser,
    chemprot_re_ner_chunk_filter,
    chemprot_re_Model
])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_chemprot_clinical download started this may take some time.
Approximate size to download 15.3 MB
[ | ][OK!]
redl_chemprot_biobert download started this may take some time.
Approximate size to download 383.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
text='''
In this study, we examined the effects of mitiglinide on various cloned K(ATP) channels (Kir6.2/SUR1, Kir6.2/SUR2A, and Kir6.2/SUR2B) reconstituted in COS-1 cells, and compared them to another meglitinide-related compound, nateglinide. Patch-clamp analysis using inside-out recording configuration showed that mitiglinide inhibits the Kir6.2/SUR1 channel currents in a dose-dependent manner (IC50 value, 100 nM) but does not significantly inhibit either Kir6.2/SUR2A or Kir6.2/SUR2B channel currents even at high doses (more than 10 microM). Nateglinide inhibits Kir6.2/SUR1 and Kir6.2/SUR2B channels at 100 nM, and inhibits Kir6.2/SUR2A channels at high concentrations (1 microM). Binding experiments on mitiglinide, nateglinide, and repaglinide to SUR1 expressed in COS-1 cells revealed that they inhibit the binding of [3H]glibenclamide to SUR1 (IC50 values: mitiglinide, 280 nM; nateglinide, 8 microM; repaglinide, 1.6 microM), suggesting that they all share a glibenclamide binding site. The insulin responses to glucose, mitiglinide, tolbutamide, and glibenclamide in MIN6 cells after chronic mitiglinide, nateglinide, or repaglinide treatment were comparable to those after chronic tolbutamide and glibenclamide treatment. These results indicate that, similar to the sulfonylureas, mitiglinide is highly specific to the Kir6.2/SUR1 complex, i.e., the pancreatic beta-cell K(ATP) channel, and suggest that mitiglinide may be a clinically useful anti-diabetic drug.
'''
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df = get_relations_df (annotations)

rel_df[rel_df.entity1!=rel_df.entity2]

Out[71]:

relation 
 entity1 
 entity1_begin 
 entity1_end 
 chunk1 
 entity2 
 entity2_begin 
 entity2_end 
 chunk2 
 confidence 
 
 
 
 
 0 
 CPR:2 
 CHEMICAL 
 43 
 53 
 mitiglinide 
 GENE-N 
 80 
 87 
 channels 
 0.99913484 
 
 
 1 
 CPR:2 
 GENE-N 
 80 
 87 
 channels 
 CHEMICAL 
 224 
 234 
 nateglinide 
 0.99174994 
 
 
 4 
 CPR:2 
 CHEMICAL 
 706 
 716 
 mitiglinide 
 GENE-Y 
 751 
 754 
 SUR1 
 0.9985078 
 
 
 6 
 CPR:2 
 CHEMICAL 
 719 
 729 
 nateglinide 
 GENE-Y 
 751 
 754 
 SUR1 
 0.9977102 
 
 
 7 
 CPR:2 
 CHEMICAL 
 736 
 746 
 repaglinide 
 GENE-Y 
 751 
 754 
 SUR1 
 0.9968532 
 
 
 8 
 CPR:2 
 CHEMICAL 
 823 
 839 
 [3H]glibenclamide 
 GENE-Y 
 844 
 847 
 SUR1 
 0.9966983 
 
 
 11 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1019 
 1025 
 glucose 
 0.9810037 
 
 
 12 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1028 
 1038 
 mitiglinide 
 0.9636817 
 
 
 13 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1041 
 1051 
 tolbutamide 
 0.98104066 
 
 
 14 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1058 
 1070 
 glibenclamide 
 0.9880949 
 
 
 15 
 CPR:2 
 GENE-N 
 998 
 1004 
 insulin 
 CHEMICAL 
 1100 
 1110 
 mitiglinide 
 0.9880691 
 
 
 34 
 CPR:2 
 CHEMICAL 
 1290 
 1300 
 mitiglinide 
 GENE-N 
 1387 
 1393 
 channel 
 0.98562336

# Merging Multiple RE Model Results

We can merge multiple RE model results by using `AnnotationMerger()`. <br/>
Now, we will build a pipeline consisting of `posology_re`, `re_ade_clinical` models and `AnnotationMerger()` to merge these RE models' results.

In [0]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

pos_ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_pos")

pos_ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_pos"])\
    .setOutputCol("pos_ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

pos_reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "pos_ner_chunks", "dependencies"])\
    .setOutputCol("pos_relations")\
    .setMaxSyntacticDistance(4)

ade_ner_tagger = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ade_ner_tags")  

ade_ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ade_ner_tags"])\
    .setOutputCol("ade_ner_chunks")

ade_reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ade_ner_chunks", "dependencies"])\
    .setOutputCol("ade_relations")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])

annotation_merger = AnnotationMerger()\
    .setInputCols("ade_relations", "pos_relations")\
    .setInputType("category")\
    .setOutputCol("all_relations")

merger_pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    pos_ner_tagger,
    pos_ner_chunker,
    dependency_parser,
    pos_reModel,
    ade_ner_tagger,
    ade_ner_chunker,
    ade_reModel,
    annotation_merger
])

empty_df= spark.createDataFrame([[""]]).toDF("text")
merger_model= merger_pipeline.fit(empty_df)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
re_ade_clinical download started this may take some time.
Approximate size to download 10.9 MB
[ | ][OK!]

In [0]:
text = """
The patient was prescribed 1 unit of naproxen for 5 days after meals for chronic low back pain. The patient was also given 1 unit of oxaprozin daily for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.. 
"""
data = spark.createDataFrame([[text]]).toDF("text")

result = merger_model.transform(data)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentences| tokens| embeddings| pos_tags| ner_pos| pos_ner_chunks| dependencies| pos_relations| ade_ner_tags| ade_ner_chunks| ade_relations| all_relations|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
\nThe patient was...|[{document, 0, 26...|[{document, 1, 95...|[{token, 1, 3, Th...|[{word_embeddings...|[{pos, 1, 3, DD, ...|[{named_entity, 1...|[{chunk, 28, 33, ...|[{dependency, 1, ...|[{category, 28, 4...|[{named_entity, 1...|[{chunk, 38, 45, ...|[{category, 134, ...|[{category, 134, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip('pos_relations.result', 'pos_relations.metadata','ade_relations.result', 'ade_relations.metadata','all_relations.result', 'all_relations.metadata')).alias("cols")) \
.select(
        F.expr("cols['0']").alias("pos_relations"),\
        F.expr("cols['1']['entity1']").alias("pos_relations_entity1"),\
        F.expr("cols['1']['chunk1']" ).alias("pos_relations_chunk1" ),\
        F.expr("cols['1']['entity2']").alias("pos_relations_entity2"),\
        F.expr("cols['1']['chunk2']" ).alias("pos_relations_chunk2" ),\
        F.expr("cols['2']").alias("ade_relations"),\
        F.expr("cols['3']['entity1']").alias("ade_relations_entity1"),\
        F.expr("cols['3']['chunk1']" ).alias("ade_relations_chunk1" ),\
        F.expr("cols['3']['entity2']").alias("ade_relations_entity2"),\
        F.expr("cols['3']['chunk2']" ).alias("ade_relations_chunk2" ),\
        F.expr("cols['4']").alias("all_relations"),\
        F.expr("cols['5']['entity1']").alias("all_relations_entity1"),\
        F.expr("cols['5']['chunk1']" ).alias("all_relations_chunk1" ),\
        F.expr("cols['5']['entity2']").alias("all_relations_entity2"),\
        F.expr("cols['5']['chunk2']" ).alias("all_relations_chunk2" )
        )

result_df.show()

+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
| pos_relations|pos_relations_entity1|pos_relations_chunk1|pos_relations_entity2|pos_relations_chunk2|ade_relations|ade_relations_entity1|ade_relations_chunk1|ade_relations_entity2|ade_relations_chunk2| all_relations|all_relations_entity1|all_relations_chunk1|all_relations_entity2|all_relations_chunk2|
+--------------+---------------------+--------------------+---------------------+--------------------+-------------+---------------------+--------------------+---------------------+--------------------+--------------+---------------------+--------------------+---------------------+--------------------+
|   DOSAGE-DRUG|               DOSAGE|              1 unit|                 DRUG|       

Now, we will build a LightPipeline and visualize the results by using `RelationExtractionVisualizer`

In [0]:
lp_merger = LightPipeline(merger_model)
lp_res = lp_merger.fullAnnotate(text)[0]

In [0]:
from sparknlp_display import RelationExtractionVisualizer


visualizer = RelationExtractionVisualizer()
vis = visualizer.display(lp_res, 'all_relations', show_relations=True, return_html=True) # default show_relations: True

displayHTML(vis)

The patient was prescribed 1 unit DOSAGE of naproxen DRUG for 5 days DURATION after meals for chronic low back pain. The patient was also given 1 unit DOSAGE of oxaprozin DRUG daily FREQUENCY for rheumatoid arthritis presented with tense bullae ADE and cutaneous fragility on the face and the back of the hands ADE .. <polyline fill="none" points="620.0,155.0 619.112244897959,153.7960798646822 618.2244897959182,152.59312871337622 617.3367346938776,151.39211553009378 616.4489795918367,150.19400929884657 615.5612244897959,148.99977900364644 614.6734693877551,147.81039362850512 613.7857142857144,146.62682215743445 612.8979591836735,145.45003357444605 612.0102040816327,144.28099686355176 611.1224489795918,143.12068100876337 610.234693877551,141.97005499409258 609.34693877551,140.83008780355124 608.4591836734694,139.70174842115105 607.5714285714286,138.5860058309038 606.6836734693879,137.48382901682123 605.795918367347,136.39618696291512 604.9081632653061,135.32404865319725 604.0204081632653,134.2683830716793 603.1326530612245,133.23015920237316 602.2448979591836,132.2103460292905 601.3571428571428,131.20991253644314 600.469387755102,130.22982770784282 599.5816326530612,129.2710605275013 598.6938775510205,128.33457997943034 597.8061224489795,127.42135504764173 596.9183673469388,126.53235471614718 596.030612244898,125.6685479689585 595.1428571428572,124.83090379008749 594.2551020408164,124.02039116354581 593.3673469387755,123.2379790733453 592.4795918367347,122.48463650349771 591.5918367346939,121.76133243801479 590.704081632653,121.06903586090831 589.8163265306123,120.40871575619002 588.9285714285714,119.78134110787173 588.0408163265306,119.18788089996517 587.1530612244899,118.62930411648207 586.265306122449,118.10657974143427 585.3775510204082,117.62067675883348 584.4897959183673,117.17256415269148 583.6020408163265,116.76321090702004 582.7142857142858,116.39358600583091 581.8265306122448,116.06465843313586 580.9387755102041,115.77739717294665 580.0510204081633,115.53277120927505 579.1632653061224,115.33174952613282 578.2755102040817,115.17530110753174 577.3877551020408,115.06439493748353 576.5,115.0 576.5,115.0 575.612244897959,114.9827622844223 574.7244897959183,115.01203580140927 573.8367346938776,115.08685156694916 572.9489795918367,115.20624059703015 572.0612244897959,115.36923390764053 571.173469387755,115.57486251476851 570.2857142857144,115.82215743440234 569.3979591836735,116.11014968253025 568.5102040816327,116.43787027514047 567.6224489795918,116.80435022822121 566.7346938775509,117.20862055776078 565.8469387755101,117.64971227974738 564.9591836734694,118.12665641016923 564.0714285714286,118.63848396501459 563.1836734693879,119.18422596027166 562.295918367347,119.7629134119287 561.4081632653061,120.37357733597398 560.5204081632653,121.01524874839565 559.6326530612245,121.68695866518203 558.7448979591835,122.3877381023213 557.8571428571428,123.11661807580174 556.969387755102,123.87262960161156 556.0816326530612,124.654803695739 555.1938775510205,125.46217137417233 554.3061224489796,126.29376365289971 553.4183673469388,127.14861154790944 552.530612244898,128.02574607518974 551.6428571428572,128.92419825072886 550.7551020408164,129.842999090515 549.8673469387755,130.7811796105364 548.9795918367347,131.73777082678137 548.0918367346939,132.71180375523804 547.204081632653,133.7023094118947 546.3163265306123,134.7083188127396 545.4285714285714,135.72886297376095 544.5408163265306,136.76297291094698 543.6530612244898,137.80967964028594 542.7653061224489,138.86801417776604 541.8775510204082,139.9370075393756 540.9897959183675,141.01569074110276 540.1020408163265,142.1030947989358 539.2142857142858,143.19825072886297 538.3265306122448,144.30018954687245 537.4387755102041,145.40794226895255 536.5510204081633,146.52053991109145 535.6632653061224,147.63701348927742 534.7755102040817,148.75639401949866 533.8877551020408,149.87771251774345 533.0,151.0" stroke="slategray" stroke-width="1" /> DRUG-FREQUENCY <polyline fill="none" points="4

# Clinical Knowledge Graph Creation using Neo4j

We have recently published a [Medium article](https://medium.com/spark-nlp/creating-knowledge-graph-by-spark-nlp-neo4j-9d18706aa08b) explains "How to create a  clinical knowledge graph using Spark NLP and Neo4j." and for detailed information, you can play with [the notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/10.2.Clinical_RE_Knowledge_Graph_with_Neo4j.ipynb). Briefly, in this article, we built a Knowledge Graph (KG) using Spark NLP Relation Extraction (RE) Models and Neo4j. Some of the Spark NLP users ask "How can I use the RE results?". First, you can use the results to feed downstream pipelines. Secondly, you can create a KG to try to get insights. The article explains how to accomplish the second option. Here are some snapshots:

# Relation Extraction Model-NER Model-Relation Pairs Table

In the table below, you can find the Relationship Extraction models and the most appropriate NER models and Relationship Pairs that can be used to get the most efficient results when using these models. <br/>
Also, you can reach this table in [Sparknlp for Healthcare Documentation](https://nlp.johnsnowlabs.com/docs/en/best_practices_pretrained_models#relation-extraction-models-and-relation-pairs-table)

|    | RELATION EXTRACTION MODEL            | RELATION EXTRACTION MODEL LABELS                                        | NER MODEL                         | RELATION PAIRS                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|---:|:-------------------------------------|:------------------------------------------------------------------------|:----------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | re_bodypart_proceduretest            | 0-1                                                                     | ner_jsl                           | ["external_body_part_or_region-test", <br/> "test-external_body_part_or_region",<br/>"internal_organ_or_component-test",<br/>"test-internal_organ_or_component",<br/>"external_body_part_or_region-procedure",<br/>"procedure-external_body_part_or_region",<br/>"procedure-internal_organ_or_component",<br/>"internal_organ_or_component-procedure"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  1 | re_ade_clinical                      | 0-1                                                                     | ner_ade_clinical                  | ["ade-drug",<br/> "drug-ade"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|  2 | redl_chemprot_biobert                | CPR:1, CPR:2, CPR:3, CPR:4, CPR:5, CPR:6, CPR:7, CPR:8, CPR:9, CPR:10   | ner_chemprot_clinical             |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  3 | re_human_phenotype_gene_clinical     | 0-1                                                                     | ner_human_phenotype_gene_clinical |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  4 | re_bodypart_directions               | 0-1                                                                     | ner_jsl                           | ["direction-external_body_part_or_region",<br/>"external_body_part_or_region-direction",<br/>"direction-internal_organ_or_component",<br/>"internal_organ_or_component-direction"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  5 | re_bodypart_problem                  | 0-1                                                                     | ner_jsl                           | ["internal_organ_or_component-cerebrovascular_disease", "cerebrovascular_disease-internal_organ_or_component",<br/>"internal_organ_or_component-communicable_disease", "communicable_disease-internal_organ_or_component",<br/>"internal_organ_or_component-diabetes", "diabetes-internal_organ_or_component",<br/>"internal_organ_or_component-disease_syndrome_disorder", "disease_syndrome_disorder-internal_organ_or_component",<br/>"internal_organ_or_component-ekg_findings", "ekg_findings-internal_organ_or_component",<br/>"internal_organ_or_component-heart_disease", "heart_disease-internal_organ_or_component",<br/>"internal_organ_or_component-hyperlipidemia", "hyperlipidemia-internal_organ_or_component",<br/>"internal_organ_or_component-hypertension", "hypertension-internal_organ_or_component",<br/>"internal_organ_or_component-imagingfindings", "imagingfindings-internal_organ_or_component",<br/>"internal_organ_or_component-injury_or_poisoning", "injury_or_poisoning-internal_organ_or_component",<br/>"internal_organ_or_component-kidney_disease", "kidney_disease-internal_organ_or_component",<br/>"internal_organ_or_component-oncological", "oncological-internal_organ_or_component",<br/>"internal_organ_or_component-psychological_condition", "psychological_condition-internal_organ_or_component",<br/>"internal_organ_or_component-symptom", "symptom-internal_organ_or_component",<br/>"internal_organ_or_component-vs_finding", "vs_finding-internal_organ_or_component",<br/>"external_body_part_or_region-communicable_disease", "communicable_disease-external_body_part_or_region",<br/>"external_body_part_or_region-diabetes", "diabetes-external_body_part_or_region",<br/>"external_body_part_or_region-disease_syndrome_disorder", "disease_syndrome_disorder-external_body_part_or_region",<br/>"external_body_part_or_region-hypertension", "hypertension-external_body_part_or_region",<br/>"external_body_part_or_region-imagingfindings", "imagingfindings-external_body_part_or_region",<br/>"external_body_part_or_region-injury_or_poisoning", "injury_or_poisoning-external_body_part_or_region",<br/>"external_body_part_or_region-obesity", "obesity-external_body_part_or_region",<br/>"external_body_part_or_region-oncological", "oncological-external_body_part_or_region",<br/>"external_body_part_or_region-overweight", "overweight-external_body_part_or_region",<br/>"external_body_part_or_region-symptom", "symptom-external_body_part_or_region",<br/>"external_body_part_or_region-vs_finding", "vs_finding-external_body_part_or_region"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|  6 | re_drug_drug_interaction_clinical    | DDI-advise, DDI-effect, DDI-mechanism, DDI-int, DDI-false               | ner_posology                      | ["drug-drug"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|  7 | re_clinical                          | TrIP, TrWP, TrCP, TrAP, TrAP, TeRP, TeCP, PIP                           | ner_clinical                      |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  8 | re_temporal_events_clinical          | AFTER, BEFORE, OVERLAP                                                  | ner_events_clinical               |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  9 | re_temporal_events_enriched_clinical | BEFORE, AFTER, SIMULTANEOUS, BEGUN_BY, ENDED_BY, DURING, BEFORE_OVERLAP | ner_events_clinical               |   No need to set pairs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 10 | re_test_problem_finding              | 0-1                                                                     | ner_jsl                           | ["test-cerebrovascular_disease", "cerebrovascular_disease-test",<br/>"test-communicable_disease", "communicable_disease-test",<br/>"test-diabetes", "diabetes-test",<br/>"test-disease_syndrome_disorder", "disease_syndrome_disorder-test",<br/>"test-heart_disease", "heart_disease-test",<br/>"test-hyperlipidemia", "hyperlipidemia-test",<br/>"test-hypertension", "hypertension-test",<br/>"test-injury_or_poisoning", "injury_or_poisoning-test",<br/>"test-kidney_disease", "kidney_disease-test",<br/>"test-obesity", "obesity-test",<br/>"test-oncological", "oncological-test",<br/>"test-psychological_condition", "psychological_condition-test",<br/>"test-symptom", "symptom-test",<br/>"ekg_findings-disease_syndrome_disorder", "disease_syndrome_disorder-ekg_findings",<br/>"ekg_findings-heart_disease", "heart_disease-ekg_findings",<br/>"ekg_findings-symptom", "symptom-ekg_findings",<br/>"imagingfindings-cerebrovascular_disease", "cerebrovascular_disease-imagingfindings",<br/>"imagingfindings-communicable_disease", "communicable_disease-imagingfindings",<br/>"imagingfindings-disease_syndrome_disorder", "disease_syndrome_disorder-imagingfindings",<br/>"imagingfindings-heart_disease", "heart_disease-imagingfindings",<br/>"imagingfindings-hyperlipidemia", "hyperlipidemia-imagingfindings",<br/>"imagingfindings-hypertension", "hypertension-imagingfindings",<br/>"imagingfindings-injury_or_poisoning", "injury_or_poisoning-imagingfindings",<br/>"imagingfindings-kidney_disease", "kidney_disease-imagingfindings",<br/>"imagingfindings-oncological", "oncological-imagingfindings",<br/>"imagingfindings-psychological_condition", "psychological_condition-imagingfindings",<br/>"imagingfindings-symptom", "symptom-imagingfindings",<br/>"vs_finding-cerebrovascular_disease", "cerebrovascular_disease-vs_finding",<br/>"vs_finding-communicable_disease", "communicable_disease-vs_finding",<br/>"vs_finding-diabetes", "diabetes-vs_finding",<br/>"vs_finding-disease_syndrome_disorder", "disease_syndrome_disorder-vs_finding",<br/>"vs_finding-heart_disease", "heart_disease-vs_finding",<br/>"vs_finding-hyperlipidemia", "hyperlipidemia-vs_finding",<br/>"vs_finding-hypertension", "hypertension-vs_finding",<br/>"vs_finding-injury_or_poisoning", "injury_or_poisoning-vs_finding",<br/>"vs_finding-kidney_disease", "kidney_disease-vs_finding",<br/>"vs_finding-obesity", "obesity-vs_finding",<br/>"vs_finding-oncological", "oncological-vs_finding",<br/>"vs_finding-overweight", "overweight-vs_finding",<br/>"vs_finding-psychological_condition", "psychological_condition-vs_finding",<br/>"vs_finding-symptom", "symptom-vs_finding"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 11 | re_test_result_date                  | is_finding_of, is_result_of, is_date_of, O                              | ner_jsl                           | ["test-test_result", "test_result-test",<br/>"test-date", "date-test",<br/>"test-imagingfindings", "imagingfindings-test",<br/>"test-ekg_findings", "ekg_findings-test",<br/>"date-test_result", "test_result-date",<br/>"date-imagingfindings", "imagingfindings-date",<br/>"date-ekg_findings", "ekg_findings-date"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 12 | re_date_clinical                     | 0-1                                                                     | ner_jsl                           | ["date-admission_discharge", "admission_discharge-date",<br/>"date-alcohol", "alcohol-date",<br/>"date-allergen", "allergen-date",<br/>"date-bmi", "bmi-date",<br/>"date-birth_entity", "birth_entity-date",<br/>"date-blood_pressure", "blood_pressure-date",<br/>"date-cerebrovascular_disease", "cerebrovascular_disease-date",<br/>"date-clinical_dept", "clinical_dept-date",<br/>"date-communicable_disease", "communicable_disease-date",<br/>"date-death_entity", "death_entity-date",<br/>"date-diabetes", "diabetes-date",<br/>"date-diet", "diet-date",<br/>"date-disease_syndrome_disorder", "disease_syndrome_disorder-date",<br/>"date-drug_brandname", "drug_brandname-date",<br/>"date-drug_ingredient", "drug_ingredient-date",<br/>"date-ekg_findings", "ekg_findings-date",<br/>"date-external_body_part_or_region", "external_body_part_or_region-date",<br/>"date-fetus_newborn", "fetus_newborn-date",<br/>"date-hdl", "hdl-date",<br/>"date-heart_disease", "heart_disease-date",<br/>"date-height", "height-date",<br/>"date-hyperlipidemia", "hyperlipidemia-date",<br/>"date-hypertension", "hypertension-date",<br/>"date-imagingfindings", "imagingfindings-date",<br/>"date-imaging_technique", "imaging_technique-date",<br/>"date-injury_or_poisoning", "injury_or_poisoning-date",<br/>"date-internal_organ_or_component", "internal_organ_or_component-date",<br/>"date-kidney_disease", "kidney_disease-date",<br/>"date-ldl", "ldl-date",<br/>"date-modifier", "modifier-date",<br/>"date-o2_saturation", "o2_saturation-date",<br/>"date-obesity", "obesity-date",<br/>"date-oncological", "oncological-date",<br/>"date-overweight", "overweight-date",<br/>"date-oxygen_therapy", "oxygen_therapy-date",<br/>"date-pregnancy", "pregnancy-date",<br/>"date-procedure", "procedure-date",<br/>"date-psychological_condition", "psychological_condition-date",<br/>"date-pulse", "pulse-date",<br/>"date-respiration", "respiration-date",<br/>"date-smoking", "smoking-date",<br/>"date-substance", "substance-date",<br/>"date-substance_quantity", "substance_quantity-date",<br/>"date-symptom", "symptom-date",<br/>"date-temperature", "temperature-date",<br/>"date-test", "test-date",<br/>"date-test_result", "test_result-date",<br/>"date-total_cholesterol", "total_cholesterol-date",<br/>"date-treatment", "treatment-date",<br/>"date-triglycerides", "triglycerides-date",<br/>"date-vs_finding", "vs_finding-date",<br/>"date-vaccine", "vaccine-date",<br/>"date-vital_signs_header", "vital_signs_header-date",<br/>"date-weight", "weight-date",<br/>"time-admission_discharge", "admission_discharge-time",<br/>"time-alcohol", "alcohol-time",<br/>"time-allergen", "allergen-time",<br/>"time-bmi", "bmi-time",<br/>"time-birth_entity", "birth_entity-time",<br/>"time-blood_pressure", "blood_pressure-time",<br/>"time-cerebrovascular_disease", "cerebrovascular_disease-time",<br/>"time-clinical_dept", "clinical_dept-time",<br/>"time-communicable_disease", "communicable_disease-time",<br/>"time-death_entity", "death_entity-time",<br/>"time-diabetes", "diabetes-time",<br/>"time-diet", "diet-time",<br/>"time-disease_syndrome_disorder", "disease_syndrome_disorder-time",<br/>"time-drug_brandname", "drug_brandname-time",<br/>"time-drug_ingredient", "drug_ingredient-time",<br/>"time-ekg_findings", "ekg_findings-time",<br/>"time-external_body_part_or_region", "external_body_part_or_region-time",<br/>"time-fetus_newborn", "fetus_newborn-time",<br/>"time-hdl", "hdl-time",<br/>"time-heart_disease", "heart_disease-time",<br/>"time-height", "height-time",<br/>"time-hyperlipidemia", "hyperlipidemia-time",<br/>"time-hypertension", "hypertension-time",<br/>"time-imagingfindings", "imagingfindings-time",<br/>"time-imaging_technique", "imaging_technique-time",<br/>"time-injury_or_poisoning", "injury_or_poisoning-time",<br/>"time-internal_organ_or_component", "internal_organ_or_component-time",<br/>"time-kidney_disease", "kidney_disease-time",<br/>"time-ldl", "ldl-time",<br/>"time-modifier", "modifier-time",<br/>"time-o2_saturation", "o2_saturation-time",<br/>"time-obesity", "obesity-time",<br/>"time-oncological", "oncological-time",<br/>"time-overweight", "overweight-time",<br/>"time-oxygen_therapy", "oxygen_therapy-time",<br/>"time-pregnancy", "pregnancy-time",<br/>"time-procedure", "procedure-time",<br/>"time-psychological_condition", "psychological_condition-time",<br/>"time-pulse", "pulse-time",<br/>"time-respiration", "respiration-time",<br/>"time-smoking", "smoking-time",<br/>"time-substance", "substance-time",<br/>"time-substance_quantity", "substance_quantity-time",<br/>"time-symptom", "symptom-time",<br/>"time-temperature", "temperature-time",<br/>"time-test", "test-time",<br/>"time-test_result", "test_result-time",<br/>"time-total_cholesterol", "total_cholesterol-time",<br/>"time-treatment", "treatment-time",<br/>"time-triglycerides", "triglycerides-time",<br/>"time-vs_finding", "vs_finding-time",<br/>"time-vaccine", "vaccine-time",<br/>"time-vital_signs_header", "vital_signs_header-time",<br/>"time-weight", "weight-time",<br/>"relativedate-admission_discharge", "admission_discharge-relativedate",<br/>"relativedate-alcohol", "alcohol-relativedate",<br/>"relativedate-allergen", "allergen-relativedate",<br/>"relativedate-bmi", "bmi-relativedate",<br/>"relativedate-birth_entity", "birth_entity-relativedate",<br/>"relativedate-blood_pressure", "blood_pressure-relativedate",<br/>"relativedate-cerebrovascular_disease", "cerebrovascular_disease-relativedate",<br/>"relativedate-clinical_dept", "clinical_dept-relativedate",<br/>"relativedate-communicable_disease", "communicable_disease-relativedate",<br/>"relativedate-death_entity", "death_entity-relativedate",<br/>"relativedate-diabetes", "diabetes-relativedate",<br/>"relativedate-diet", "diet-relativedate",<br/>"relativedate-disease_syndrome_disorder", "disease_syndrome_disorder-relativedate",<br/>"relativedate-drug_brandname", "drug_brandname-relativedate",<br/>"relativedate-drug_ingredient", "drug_ingredient-relativedate",<br/>"relativedate-ekg_findings", "ekg_findings-relativedate",<br/>"relativedate-external_body_part_or_region", "external_body_part_or_region-relativedate",<br/>"relativedate-fetus_newborn", "fetus_newborn-relativedate",<br/>"relativedate-hdl", "hdl-relativedate",<br/>"relativedate-heart_disease", "heart_disease-relativedate",<br/>"relativedate-height", "height-relativedate",<br/>"relativedate-hyperlipidemia", "hyperlipidemia-relativedate",<br/>"relativedate-hypertension", "hypertension-relativedate",<br/>"relativedate-imagingfindings", "imagingfindings-relativedate",<br/>"relativedate-imaging_technique", "imaging_technique-relativedate",<br/>"relativedate-injury_or_poisoning", "injury_or_poisoning-relativedate",<br/>"relativedate-internal_organ_or_component", "internal_organ_or_component-relativedate",<br/>"relativedate-kidney_disease", "kidney_disease-relativedate",<br/>"relativedate-ldl", "ldl-relativedate",<br/>"relativedate-modifier", "modifier-relativedate",<br/>"relativedate-o2_saturation", "o2_saturation-relativedate",<br/>"relativedate-obesity", "obesity-relativedate",<br/>"relativedate-oncological", "oncological-relativedate",<br/>"relativedate-overweight", "overweight-relativedate",<br/>"relativedate-oxygen_therapy", "oxygen_therapy-relativedate",<br/>"relativedate-pregnancy", "pregnancy-relativedate",<br/>"relativedate-procedure", "procedure-relativedate",<br/>"relativedate-psychological_condition", "psychological_condition-relativedate",<br/>"relativedate-pulse", "pulse-relativedate",<br/>"relativedate-respiration", "respiration-relativedate",<br/>"relativedate-smoking", "smoking-relativedate",<br/>"relativedate-substance", "substance-relativedate",<br/>"relativedate-substance_quantity", "substance_quantity-relativedate",<br/>"relativedate-symptom", "symptom-relativedate",<br/>"relativedate-temperature", "temperature-relativedate",<br/>"relativedate-test", "test-relativedate",<br/>"relativedate-test_result", "test_result-relativedate",<br/>"relativedate-total_cholesterol", "total_cholesterol-relativedate",<br/>"relativedate-treatment", "treatment-relativedate",<br/>"relativedate-triglycerides", "triglycerides-relativedate",<br/>"relativedate-vs_finding", "vs_finding-relativedate",<br/>"relativedate-vaccine", "vaccine-relativedate",<br/>"relativedate-vital_signs_header", "vital_signs_header-relativedate",<br/>"relativedate-weight", "weight-relativedate",<br/>"relativetime-admission_discharge", "admission_discharge-relativetime",<br/>"relativetime-alcohol", "alcohol-relativetime",<br/>"relativetime-allergen", "allergen-relativetime",<br/>"relativetime-bmi", "bmi-relativetime",<br/>"relativetime-birth_entity", "birth_entity-relativetime",<br/>"relativetime-blood_pressure", "blood_pressure-relativetime",<br/>"relativetime-cerebrovascular_disease", "cerebrovascular_disease-relativetime",<br/>"relativetime-clinical_dept", "clinical_dept-relativetime",<br/>"relativetime-communicable_disease", "communicable_disease-relativetime",<br/>"relativetime-death_entity", "death_entity-relativetime",<br/>"relativetime-diabetes", "diabetes-relativetime",<br/>"relativetime-diet", "diet-relativetime",<br/>"relativetime-disease_syndrome_disorder", "disease_syndrome_disorder-relativetime",<br/>"relativetime-drug_brandname", "drug_brandname-relativetime",<br/>"relativetime-drug_ingredient", "drug_ingredient-relativetime",<br/>"relativetime-ekg_findings", "ekg_findings-relativetime",<br/>"relativetime-external_body_part_or_region", "external_body_part_or_region-relativetime",<br/>"relativetime-fetus_newborn", "fetus_newborn-relativetime",<br/>"relativetime-hdl", "hdl-relativetime",<br/>"relativetime-heart_disease", "heart_disease-relativetime",<br/>"relativetime-height", "height-relativetime",<br/>"relativetime-hyperlipidemia", "hyperlipidemia-relativetime",<br/>"relativetime-hypertension", "hypertension-relativetime",<br/>"relativetime-imagingfindings", "imagingfindings-relativetime",<br/>"relativetime-imaging_technique", "imaging_technique-relativetime",<br/>"relativetime-injury_or_poisoning", "injury_or_poisoning-relativetime",<br/>"relativetime-internal_organ_or_component", "internal_organ_or_component-relativetime",<br/>"relativetime-kidney_disease", "kidney_disease-relativetime",<br/>"relativetime-ldl", "ldl-relativetime",<br/>"relativetime-modifier", "modifier-relativetime",<br/>"relativetime-o2_saturation", "o2_saturation-relativetime",<br/>"relativetime-obesity", "obesity-relativetime",<br/>"relativetime-oncological", "oncological-relativetime",<br/>"relativetime-overweight", "overweight-relativetime",<br/>"relativetime-oxygen_therapy", "oxygen_therapy-relativetime",<br/>"relativetime-pregnancy", "pregnancy-relativetime",<br/>"relativetime-procedure", "procedure-relativetime",<br/>"relativetime-psychological_condition", "psychological_condition-relativetime",<br/>"relativetime-pulse", "pulse-relativetime",<br/>"relativetime-respiration", "respiration-relativetime",<br/>"relativetime-smoking", "smoking-relativetime",<br/>"relativetime-substance", "substance-relativetime",<br/>"relativetime-substance_quantity", "substance_quantity-relativetime",<br/>"relativetime-symptom", "symptom-relativetime",<br/>"relativetime-temperature", "temperature-relativetime",<br/>"relativetime-test", "test-relativetime",<br/>"relativetime-test_result", "test_result-relativetime",<br/>"relativetime-total_cholesterol", "total_cholesterol-relativetime",<br/>"relativetime-treatment", "treatment-relativetime",<br/>"relativetime-triglycerides", "triglycerides-relativetime",<br/>"relativetime-vs_finding", "vs_finding-relativetime",<br/>"relativetime-vaccine", "vaccine-relativetime",<br/>"relativetime-vital_signs_header", "vital_signs_header-relativetime",<br/>"relativetime-weight", "weight-relativetime"] |